# Quantum Pseudo-Entropy in Post-Selected Entanglement with Noise

This notebook implements and analyzes post-selected entanglement in quantum circuits, calculating pseudo-entropy and examining the effects of noise on the measurements. It serves as the computational backend for the theoretical and methodological discussions in the main thesis, offloading detailed derivations, simulation parameters, and extensive data analysis.

## Setup and Imports

This section ensures all necessary Python packages are installed and imports the required libraries for quantum simulation (Qiskit), numerical analysis (NumPy, SciPy), data manipulation (Pandas), and visualization (Matplotlib).

In [ ]:
## Install required packages 
#%pip install -U qiskit qiskit-aer matplotlib pylatexenc scikit-learn tabulate sympy pdflatex pandas openpyxl

## Configuration Parameters

This section defines the constants and simulation parameters that govern the quantum experiments. These include the resolution of the simulated parameter space for $\beta$ (controlled interaction strength) and $\delta$ (coherent error angle), the number of measurement shots for noisy simulations, and the specific IBM `FakeBackend` used for noise modeling. While these parameters are defined here for computational execution, their detailed justification, selection criteria, and discussion regarding hardware calibration and parameter space design are provided in the main thesis and supplementary materials.

In [ ]:
# Import required libraries
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, partial_trace
from qiskit.circuit import Parameter

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from IPython.display import Markdown, display, Latex

from tabulate import tabulate

from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats import t, chisquare, beta
from scipy import ndimage

import sympy as sp

from sklearn.metrics import r2_score

import warnings

import os
import csv

import pickle

import math

import pandas as pd
from openpyxl.styles import NamedStyle
from openpyxl.worksheet.table import Table, TableStyleInfo

## Utility Functions

This section provides helper functions for displaying formatted output in the Jupyter notebook, handling numerical rounding for LaTeX compatibility, and managing the caching of computationally intensive results to disk. These utilities streamline the workflow and ensure consistent presentation of data.

In [ ]:
# Simulation Parameters

# Number of points for beta and delta sweeps (must be odd to ensure a middle point at zero).
# A square grid is used with the same number of points for both beta and delta.
# This ensures symmetry and accurate sampling around zero.
DELTA_POINTS = 401
BETA_POINTS = 1601

# Range of angles for beta and delta sweeps (both positive and negative).
# A relatively large range is selected to simulate noise effectively.
# The same range is applied to both beta and delta for consistency in simulations.
DELTA_RANGE = np.pi / 128
BETA_RANGE = np.pi




## Core Functions


## Circuit Execution and Data Collection

This section defines the functions responsible for executing quantum circuits and collecting pseudo-entropy data across the defined parameter sweeps. The `calculate_generalized_density_matrix` and `calculate_pseudo_entropy` functions are central to this process, handling the complex-valued calculations required for pseudo-entropy. The `run_circuits_and_calculate_entropy` orchestrates the simulation loop, calculating initial and final states and then computing pseudo-entropy for each $(\beta, \delta)$ pair.

In [ ]:
def display_and_log_print(text):
    """
    Replaces the print function to display the given text as Markdown in the Jupyter notebook 
    and log it to the 'results/output.txt' file.
    
    Args:
        text (str): The text to display and log.
    """
    # Display the text as Markdown in the notebook
    print(text)
    
    # Log the text to the output file
    try:
        with open('results/output.txt', 'a') as file:
            file.write(text + '\n\n')  # Add a newline after each entry for readability
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

def to_latex(expression):
    return sp.latex(expression, mul_symbol="dot", min=3, max=3)

def round_number(number):
    # Handle special cases: NaN, Inf, and Zero
    if number == 0:
        return 0
    if np.isinf(number):  # Check for Infinity
        return np.inf if number > 0 else -np.inf
    if np.isnan(number):  # Check for NaN
        return np.nan
    exponent = int(np.floor(np.log10(abs(number))))
    mantissa = number / (10 ** exponent)
    rounded_mantissa = round(mantissa, 3)
    return rounded_mantissa * (10 ** exponent)

def display_and_log_markdown(text):
    """ 
    Displays the given text as Markdown in the Jupyter Notebook 
    and logs it to the 'results/output.txt' file.
    
    Args:
        text (str): The text to display and log.
    """
    # Display the text as Markdown in the notebook
    display(Markdown(text))
    
    # Log the text to the output file
    try:
        with open('results/output.txt', 'a') as file:
            file.write(text + '\n\n')  # Add a newline after each entry for readability
    except Exception as e:
        display_and_log_print(f"An error occurred while writing to the file: {e}")




## Quantum Circuit Implementation

This section defines the quantum circuits used for initial state preparation and the measurement. The `create_initial_state_circuit` function prepares the initial state $\ket{+1}$, while `create_measurement_circuit` constructs the main two-qubit circuit including the CNOT gate and the parameterized $R_Y$, $R_Z$, and $R_X$ rotations that model coherent errors. This section provides the *detailed Qiskit implementation* of these circuits, complementing the high-level descriptions and canonical circuit diagrams presented in the main thesis. The specifics of virtual gate construction are fully expressed here.

In [ ]:
def load_or_calculate(file_path, calculate_function, *args, **kwargs):
    """
    Load a value from a file if it exists; otherwise, calculate it, save it, and return the value.

    Args:
        file_path (str): Path to the file to load/save the value.
        calculate_function (callable): Function to calculate the value if not loaded.
        *args: Positional arguments for the calculate_function.
        **kwargs: Keyword arguments for the calculate_function.

    Returns:
        Any: The loaded or calculated value.
    """
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    if os.path.exists(file_path):
        display_and_log_print(f"Loading from {file_path}...")
        with open(file_path, "rb") as f:
            value = pickle.load(f)
    else:
        display_and_log_print(f"Calculating and saving to {file_path}...")
        value = calculate_function(*args, **kwargs)
        with open(file_path, "wb") as f:
            pickle.dump(value, f)
    return value




# Core Functions


## Circuit Execution and Data Collection

This section defines the functions responsible for executing quantum circuits and collecting pseudo-entropy data across the defined parameter sweeps. The `calculate_generalized_density_matrix` and `calculate_pseudo_entropy` functions are central to this process, handling the complex-valued calculations required for pseudo-entropy. The `run_circuits_and_calculate_entropy` orchestrates the simulation loop, calculating initial and final states and then computing pseudo-entropy for each $(\beta, \delta)$ pair.

**Note on Noise Model and Tomography Overhead:** While this notebook incorporates a `FakeBackend` for noise modeling in the simulation, the *detailed noise model parameters* and the *comprehensive analysis of tomography overhead* are offloaded to the supplementary materials. This notebook focuses on the computational implementation of the pseudo-entropy calculation under noisy conditions.

In [ ]:
def calculate_generalized_density_matrix(initial_state: Statevector, final_state: Statevector, regularization) -> np.ndarray:
    """
    Calculate the generalized non-Hermitian matrix between initial and final states for post-selected measurements.
    This matrix is intentionally non-Hermitian, which is a key feature of post-selected measurements.

    Args:
        initial_state (Statevector): Initial quantum state
        final_state (Statevector): Final quantum state after measurement
        regularization (float): Regularization parameter to handle numerical instabilities
    
    Returns:
        np.ndarray: Generalized non-Hermitian matrix or None if states are invalid
    """
    if initial_state.dim != final_state.dim:
        display_and_log_print("Initial and final states have different dimensions.")
        return None

    # Check for valid state vectors
    if np.any(np.isnan(initial_state.data)) or np.any(np.isnan(final_state.data)):
        display_and_log_print("Invalid state vectors: NaN detected.")
        return None
        
    inner_product = initial_state.inner(final_state)  # Complex inner product without abs()
    
    # Use a threshold for numerical stability, but don't enforce it to be real
    if np.abs(inner_product) > regularization:
        try:
            # Construct the non-Hermitian generalized matrix
            # \hat{\rho} = \frac{|\psi_i\rangle \langle \psi_f|}{\langle \psi_f | \psi_i \rangle}
            rho_generalized = np.outer(initial_state.data, final_state.conjugate().data) / inner_product
                        
            # Perform partial trace over subsystem B (second qubit)
            rho_A = partial_trace(rho_generalized, [1])
            
            # Check for numerical instabilities but allow complex values
            if np.any(np.isnan(rho_A.data)):
                display_and_log_print("Numerical instability detected in the partial trace result.")
                return None
                
            return rho_A.data  # Return the numpy array, not the DensityMatrix object
            
        except (np.linalg.LinAlgError, ValueError) as e:
            display_and_log_print(f"Error in matrix calculation: {e}")
            return None
    else:
        display_and_log_print("Inner product is too small, returning None.")
        return None



## Analysis and Visualization Functions

This section provides a suite of functions for analyzing and visualizing the simulation results. It includes tools for calculating descriptive statistics of pseudo-entropy components, generating various heatmaps (Cartesian, polar, and sensitivity maps), and saving figures in a format suitable for LaTeX inclusion. These functions are crucial for interpreting the complex behavior of pseudo-entropy across the parameter space.

In [ ]:
def calculate_pseudo_entropy(initial_state: Statevector, final_state: Statevector, regularization=1e-8) -> tuple:
    """
    Calculate the pseudo-entropy, its standard deviation, and the trace of the reduced transition matrix
    between initial and final quantum states in a post-selected weak measurement context.

    For non-Hermitian generalized density matrices, eigenvalues can be complex, leading to complex
    pseudo-entropy values. The standard deviation is computed via the complex variance using:
        |z|² = (Re(z))² + (Im(z))².

    Args:
        initial_state (Statevector): Initial quantum state.
        final_state (Statevector): Final quantum state after post-selection.
        regularization (float): Threshold to avoid log(0) and division errors.

    Returns:
        tuple:
            - pseudo_entropy (complex): May be complex due to non-Hermitian input.
            - std_deviation (float): Real-valued uncertainty estimate.
            - trace_rho (complex): Trace of the reduced (unnormalized) transition matrix.
    """
    rho_generalized = calculate_generalized_density_matrix(initial_state, final_state, regularization)
    
    if rho_generalized is None:
        display_and_log_print("Failed to calculate generalized density matrix; returning (0, 0, 0).")
        return np.nan, np.nan, np.nan

    trace_rho = np.trace(rho_generalized)

    # Use eigenvalue decomposition that supports non-Hermitian matrices
    eigenvalues = np.linalg.eigvals(rho_generalized)

    with np.errstate(divide='ignore', invalid='ignore'):
        log_eigenvalues = np.zeros_like(eigenvalues, dtype=complex)
        non_zero = np.abs(eigenvalues) > regularization
        log_eigenvalues[non_zero] = np.log2(eigenvalues[non_zero])

        num_zero = np.sum(~non_zero)

        if np.all(~non_zero):
            # All eigenvalues are nonzero; nothing to report.
            pass
        elif num_zero == 1:
            # One eigenvalue is zero (or below threshold); this is expected for a pure state. No warning needed.
            pass
        elif num_zero > 1:
            display_and_log_print(f"WARNING: {num_zero} eigenvalues are zero (or below threshold); pseudo-entropy may be ill-defined or degenerate.")

        # Optionally, still check for NaN or Inf in log_eigenvalues
        if np.any(np.isnan(log_eigenvalues)):
            display_and_log_print("WARNING: NaN encountered in log_eigenvalues!")
        if np.any(np.isinf(log_eigenvalues)):
            display_and_log_print("WARNING: Inf encountered in log_eigenvalues!")


    # Calculate pseudo-entropy (can be complex)
    pseudo_entropy = -np.sum(eigenvalues * log_eigenvalues)

    # Reconstruct (log ρ)² using eigendecomposition
    eigvecs = np.linalg.eig(rho_generalized)[1]
    log_rho_squared = eigvecs @ np.diag(log_eigenvalues**2) @ np.linalg.inv(eigvecs)

    # Expectation value of (log ρ)²
    log_rho_squared_expectation = np.trace(log_rho_squared @ rho_generalized)

    # Complex variance and real standard deviation
    variance = log_rho_squared_expectation - pseudo_entropy**2
    std_deviation = np.sqrt(np.real(variance)**2 + np.imag(variance)**2)

    return pseudo_entropy, std_deviation, trace_rho

## Quantum Circuit Implementation
Define the quantum circuit for post-selected measurements

## Execute Experiments

This section describes how the quantum circuits are executed and how the pseudo-entropy data is collected across the defined parameter sweeps. The `calculate_initial_state` and `calculate_final_state` functions prepare the necessary quantum states for pseudo-entropy computation. The `run_circuits_and_calculate_entropy` function then orchestrates the simulation loop, calculating pseudo-entropy and its standard deviation for each $(\beta, \delta)$ pair.

In [ ]:
def create_initial_state_circuit():
    """
    Creates a parameterized circuit for the initial state preparation.
    Returns:
        QuantumCircuit: Circuit with Hadamard and X gates
    """
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.x(1)
    circuit.barrier(label='Initial State (|+1>)')
    return circuit

def calculate_initial_state(noise_model):
    """
    Calculate the initial state with a rotation.

    Args:
        noise_model (NoiseModel): Quantum noise model
    
    Returns:
        Statevector: The initial state
    """
    initial_circuit = create_initial_state_circuit()
    
    simulator = Aer.get_backend('statevector_simulator')
    transpiled_circuit = transpile(initial_circuit, simulator)
    job = simulator.run(transpiled_circuit, noise_model=noise_model, shots=1)
    result = job.result()
    
    return Statevector(result.get_statevector())



## Results Analysis

This section focuses on visualizing and interpreting the results of the quantum simulations. It includes functions for plotting the real, imaginary, magnitude, and phase components of the pseudo-entropy, providing a comprehensive view of its behavior across the parameter space. This section generates the full phase diagram plots, which are summarized and discussed in the main thesis.

In [ ]:
def create_measurement_circuit():
    """
    Creates a parameterized circuit for the measurement.

    Returns:
        QuantumCircuit: Parameterized circuit with beta and delta rotations
    """
    beta = Parameter('$\\beta$')
    delta = Parameter('$\\delta$')
    circuit = QuantumCircuit(2)
    
    circuit.barrier(label='Group 1: Entanglement')
    circuit.cx(0, 1)

    circuit.barrier(label='Group 2: Virtual Gates')
    circuit.ry(delta, 1)
    circuit.rz(beta+delta, 1)
    circuit.rx(beta, 1)    

    circuit.barrier(label='Group 3: Final State')
    circuit.rz(np.pi/2, 1)

    return circuit
def calculate_final_state(beta, initial_state, noise_model, delta):
    """
    Calculate the final state after entanglement.

    Args:
        beta (float): Angle for Ry and Rz rotations
        initial_state (Statevector): The initial state of the system
        noise_model (NoiseModel): Quantum noise model
        delta (float): Angle for additional Rx rotation (coherent error)
    
    Returns:
        Statevector: The final state
    """
    measurement_circuit = create_measurement_circuit()
    
    # Initialize with the provided initial state
    full_circuit = QuantumCircuit(2)
    full_circuit.initialize(initial_state, [0, 1])
    full_circuit = full_circuit.compose(measurement_circuit)
    
    # Assign parameters for beta and delta
    assigned_circuit = full_circuit.assign_parameters({
        measurement_circuit.parameters[0]: beta,
        measurement_circuit.parameters[1]: delta
    })
    
    simulator = Aer.get_backend('statevector_simulator')
    transpiled_circuit = transpile(assigned_circuit, simulator)
    job = simulator.run(transpiled_circuit, noise_model=noise_model, shots=1)
    result = job.result()
    final_state = Statevector(result.get_statevector())
    
    return final_state



## Classical vs Quantum Behavior Analysis

This section focuses on identifying and visualizing regions where pseudo-entropy exhibits "classical-like" behavior versus "quantum" behavior. The `plot_classical_like_quantum` function applies a threshold to the imaginary component of pseudo-entropy to classify regions, generating heatmaps that visually distinguish these areas. It also provides statistical summaries of the proportion of classical-like, quantum, and undefined (NaN) points.

In [ ]:
def run_circuits_and_calculate_entropy(betas, deltas, noise_model):
    """
    Execute quantum circuits and calculate pseudo-entropy and standard deviation for given parameters.
    
    Args:
        betas (array): Array of angles for Ry and Rz rotations
        deltas (array): Array of angles for additional Rx rotation (coherent error)
        noise_model (NoiseModel): Quantum noise model
    
    Returns:
        tuple: (pseudo_entropy, std_deviation) where each is a 2D array with shape (len(deltas), len(betas))
    """
    pseudo_entropy = np.full((len(deltas), len(betas)), np.nan, dtype=complex)
    std_deviation = np.full((len(deltas), len(betas)), np.nan)
    traces_rho = np.full((len(deltas), len(betas)), np.nan, dtype=complex)
    
    initial_state = calculate_initial_state(noise_model)
    
    for j, beta in enumerate(betas):
        for i, delta in enumerate(deltas):
            try:
                final_state = calculate_final_state(beta, initial_state, noise_model, delta)
                entropy_value, std_value, trace_rho = calculate_pseudo_entropy(initial_state, final_state)
                if np.isnan(entropy_value) or np.isnan(std_value) or np.isnan(trace_rho):
                    display_and_log_markdown(fr"Calculated values contain NaN, indicating a numerical issue in the pseudo-entropy calculation - $\beta/\pi$:{beta/np.pi}, $\delta/\pi$:{delta/np.pi}$")
                    continue
                # Store the results in the respective arrays
                pseudo_entropy[i, j] = entropy_value
                std_deviation[i, j] = std_value
                traces_rho[i, j] = trace_rho
            except Exception as e:
                display_and_log_print(f"Error at beta {beta}, delta {delta}: {e}")

    return pseudo_entropy, std_deviation, traces_rho





## Analysis and Visualization Functions
Define functions for analyzing and plotting results

## Sensitivity Analysis

This section quantifies how changes in the $\beta$ and $\delta$ parameters affect the pseudo-entropy. It includes functions to calculate sensitivity maps (inverse of the gradient magnitude), which visually highlight regions of high and low responsiveness of pseudo-entropy to parameter variations. These maps are crucial for identifying optimal operating points for error detection and calibration. The detailed sub-subsection analysis of sensitivity methods is offloaded to the supplementary materials.

In [ ]:
def process_circuit(circuit, filename_prefix, display_name, results_dir="results"):
    """
    Process a single circuit: display as markdown, save PNG and LaTeX.
    
    Args:
        circuit: QuantumCircuit object to process
        filename_prefix: Prefix for saved files
        display_name: Name to display in markdown
        results_dir: Directory to save files
    
    Returns:
        The processed circuit
    """
    display_and_log_print(f"\n## {display_name}")
    
    # Display circuit as markdown (text representation)
    circuit_text = str(circuit.draw(output='text'))
    display_and_log_print("```")
    display_and_log_print(circuit_text)
    display_and_log_print("```")
    
    # Save circuit as PNG image
    try:
        fig = circuit.draw(output='mpl', style='textbook')
        png_path = os.path.join(results_dir, f'{filename_prefix}.png')
        fig.savefig(png_path, dpi=300, bbox_inches='tight')
        display_and_log_print(f"Circuit saved as: {png_path}")
        plt.close(fig)
    except Exception as e:
        display_and_log_print(f"Error saving {filename_prefix} PNG: {e}")
    
    # Save LaTeX source
    try:
        latex_path = os.path.join(results_dir, f'{filename_prefix}.tex')
        with open(latex_path, 'w') as f:
            f.write(circuit.draw(output='latex_source'))
        display_and_log_print(f"LaTeX source saved as: {latex_path}")
    except Exception as e:
        display_and_log_print(f"Error saving {filename_prefix} LaTeX: {e}")
    
    return circuit


def create_concatenated_circuit(initial_circuit, measurement_circuit):
    """
    Create a concatenated circuit from initial and measurement circuits.
    
    Args:
        initial_circuit: QuantumCircuit for initial state preparation
        measurement_circuit: QuantumCircuit for measurement operations
    
    Returns:
        QuantumCircuit: Concatenated circuit with barriers
    """
    display_and_log_print("\n## Creating Concatenated Circuit")
    
    # Determine maximum dimensions needed
    max_qubits = max(initial_circuit.num_qubits, measurement_circuit.num_qubits)
    max_clbits = max(initial_circuit.num_clbits, measurement_circuit.num_clbits)
    
    # Create new circuit with unified register structure
    concatenated_circuit = QuantumCircuit(max_qubits, max_clbits)
    concatenated_circuit.name = "Concatenated Circuit (Initial + Measurement)"
    
    # Add barriers for clarity and compose circuits
    concatenated_circuit.barrier()
    concatenated_circuit.compose(initial_circuit, inplace=True)
    concatenated_circuit.barrier()
    concatenated_circuit.compose(measurement_circuit, inplace=True)
    concatenated_circuit.barrier()
    
    return concatenated_circuit


def display_all_circuits():
    """
    Creates and returns all circuits for visualization using markdown rendering.
    Saves rendered circuits to results/ folder as PNG and LaTeX source.
    """
    results_dir = "results"
    
    # Create the individual circuits
    initial_circuit = create_initial_state_circuit()
    measurement_circuit = create_measurement_circuit()
    
    # Set circuit names
    initial_circuit.name = "Initial State Circuit"
    measurement_circuit.name = "Pseudo-Entropy Measurement Circuit"
    
    # Process each circuit
    display_and_log_print("# Quantum Circuit Visualization")
    
    initial_circuit = process_circuit(
        initial_circuit, 
        'initial_state_circuit', 
        'Initial State Circuit',
        results_dir
    )
    
    measurement_circuit = process_circuit(
        measurement_circuit, 
        'measurement_circuit', 
        'Measurement Circuit',
        results_dir
    )
    
    # Create and process concatenated circuit
    concatenated_circuit = create_concatenated_circuit(initial_circuit, measurement_circuit)
    concatenated_circuit = process_circuit(
        concatenated_circuit, 
        'concatenated_circuit', 
        'Concatenated Circuit',
        results_dir
    )
    
    # Create comprehensive summary file
    try:
        summary_content = f"""# Quantum Circuit Summary

## Circuit Statistics:
- **Initial State Circuit**: {initial_circuit.num_qubits} qubits, depth {initial_circuit.depth()}
- **Measurement Circuit**: {measurement_circuit.num_qubits} qubits, depth {measurement_circuit.depth()}
- **Concatenated Circuit**: {concatenated_circuit.num_qubits} qubits, depth {concatenated_circuit.depth()}

## Generated Files:
- `initial_state_circuit.png` - Initial state circuit diagram
- `initial_state_circuit.tex` - Initial state circuit LaTeX source
- `measurement_circuit.png` - Measurement circuit diagram  
- `measurement_circuit.tex` - Measurement circuit LaTeX source
- `concatenated_circuit.png` - Concatenated circuit diagram
- `concatenated_circuit.tex` - Concatenated circuit LaTeX source

## Operation Counts:

### Initial State Circuit:
{dict(initial_circuit.count_ops())}

### Measurement Circuit:
{dict(measurement_circuit.count_ops())}

### Concatenated Circuit:
{dict(concatenated_circuit.count_ops())}
"""
        
        summary_path = os.path.join(results_dir, 'circuit_summary.md')
        with open(summary_path, 'w') as f:
            f.write(summary_content)
        display_and_log_print(f"\nSummary saved as: {summary_path}")
    except Exception as e:
        display_and_log_print(f"Error saving summary: {e}")
    
    display_and_log_print(f"\n✅ All circuits processed and saved to '{results_dir}/' folder")
    
    return {
        'initial_circuit': initial_circuit,
        'measurement_circuit': measurement_circuit,
        'concatenated_circuit': concatenated_circuit
    }

In [ ]:
def apply_excel_formatting(worksheet, df):
    """
    Apply Excel formatting to the worksheet based on column content and values.
    Also formats the data as an Excel table.
    
    Args:
        worksheet: openpyxl worksheet object
        df (pd.DataFrame): DataFrame containing the data
        
    Returns:
        None
    """    
    # Create named styles for different formatting needs
    workbook = worksheet.parent
    
    # Create and register styles (skip if already exists)
    styles_to_create = [
        ("scientific", '0.00E+00'),
        ("two_decimal", '0.00'),
        ("percentage", '0.00%'),
        ("percentage_append", '0.00"%"'),
        ("percentage_scientific", '0.00E+00'),
        ("integer", '0')
    ]
    
    for style_name, number_format in styles_to_create:
        try:
            style = NamedStyle(name=style_name)
            style.number_format = number_format
            workbook.add_named_style(style)
        except ValueError:
            # Style already exists, skip
            pass
    
    # Apply formatting to each column
    for col_idx, column_name in enumerate(df.columns, 1):
        for row_idx in range(2, len(df) + 2):  # Start from row 2 (after header)
            cell = worksheet.cell(row=row_idx, column=col_idx)
            value = df.iloc[row_idx - 2, col_idx - 1]
            
            if pd.isna(value):
                continue
                
            # Apply formatting based on column name and value
            if column_name == 'Length (Cells)':
                # Integer formatting for Length column
                cell.style = "integer"
            elif column_name == 'Threshold':
                # Threshold column: percentage unless < 10^-3, then scientific
                if abs(value) < 1e-3:
                    cell.style = "percentage_scientific"
                else:
                    cell.style = "percentage"
            elif '(%)' in column_name:
                # Percentage columns: display as percentage
                cell.style = "percentage_append"
            elif isinstance(value, (int, float)):
                # Other numeric columns: scientific if < 0.01 or > 1000, otherwise 2 decimals
                if abs(value) < 0.01 or abs(value) > 1000:
                    cell.style = "scientific"
                else:
                    cell.style = "two_decimal"
    
    # Format as Excel table if there's data
    if len(df) > 0:
        # Define table range (including headers)
        table_range = f"A1:{chr(64 + len(df.columns))}{len(df) + 1}"
        
        # Create table with a unique name based on worksheet name
        table_name = f"Table_{worksheet.title}"
        table = Table(displayName=table_name, ref=table_range)
        
        # Add table style
        style = TableStyleInfo(
            name="TableStyleMedium9", 
            showFirstColumn=False,
            showLastColumn=False, 
            showRowStripes=True, 
            showColumnStripes=False
        )
        table.tableStyleInfo = style
        
        # Add the table to the worksheet
        worksheet.add_table(table)




In [ ]:
def calculate_complex_statistics(data):
    """
    Calculate statistics for complex data including real, imaginary, magnitude, and phase parts.
    Ignores NaN values in all calculations.

    Args:
        data (complex ndarray): 2D array of complex values

    Returns:
        dict: Dictionary containing statistics for all components
    """
    # Standard statistics with NaN handling
    stats = {
        'real': {
            'max': np.nanmax(data.real),
            'avg': np.nanmean(data.real),
            'median': np.nanmedian(data.real),
            'min': np.nanmin(data.real),
            'std': np.nanstd(data.real),
        },
        'imaginary': {
            'max': np.nanmax(data.imag),
            'avg': np.nanmean(data.imag),
            'median': np.nanmedian(data.imag),
            'min': np.nanmin(data.imag),
            'std': np.nanstd(data.imag), 
        },
        'magnitude': {
            'max': np.nanmax(np.abs(data)),
            'avg': np.nanmean(np.abs(data)),
            'median': np.nanmedian(np.abs(data)),
            'min': np.nanmin(np.abs(data)),
            'std': np.nanstd(np.abs(data)),
        },
        'phase': {
            'max': np.nanmax(np.angle(data, deg=False) / np.pi),
            'avg': np.nanmean(np.angle(data, deg=False) / np.pi),
            'median': np.nanmedian(np.angle(data, deg=False) / np.pi),
            'min': np.nanmin(np.angle(data, deg=False) / np.pi),
            'std': np.nanstd(np.angle(data, deg=False) / np.pi),
        },
    }

    return stats

def statistics_to_dataframe(stats_dict, data_type):
    """
    Converts the statistics dictionary into a pandas DataFrame.
    """
    df_data = []
    for stat_name in ['max', 'avg', 'median', 'min', 'std']:
        row = {
            'Statistic': stat_name.capitalize(),
            'Real Part': stats_dict['real'][stat_name],
            'Imaginary Part': stats_dict['imaginary'][stat_name],
            'Magnitude': stats_dict['magnitude'][stat_name],
            'Phase/π': stats_dict['phase'][stat_name],
            'Data Type': data_type
        }
        df_data.append(row)
    return pd.DataFrame(df_data)

def export_statistics_to_excel(stats_df, filename='results/statistics_results.xlsx'):
    """
    Exports a pandas DataFrame of statistics to an Excel file, appending to existing sheets
    or creating new ones.
    """
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    writer = pd.ExcelWriter(filename, engine='openpyxl')

    sheet_name = stats_df['Data Type'].iloc[0] if not stats_df.empty else 'Sheet1'
    
    # If the sheet already exists, remove it to prevent duplicate headers
    if sheet_name in writer.book.sheetnames:
        idx = writer.book.sheetnames.index(sheet_name)
        writer.book.remove(writer.book.worksheets[idx])
    
    stats_df.to_excel(writer, sheet_name=sheet_name, index=False)
    apply_excel_formatting(writer.sheets[sheet_name], stats_df) # Apply formatting
    
    writer.close() # Use close() instead of save() for openpyxl engine
    display_and_log_print(f"Statistics data exported to {filename} (Sheet: {sheet_name})")


def display_statistics_table(stats, data_type, data):
    """
    Display calculated statistics in a formatted Markdown table, including middle value and required shots.
    Also exports the statistics to an Excel file.

    Args:
        stats (dict): Statistics dictionary from calculate_complex_statistics.
        data_type (str): Description of the analyzed data type.
        data (complex ndarray): Original 2D array of complex values to extract middle point.
    """
    # Calculate middle indices
    mid_row = data.shape[0] // 2
    mid_col = data.shape[1] // 2
    mid_value = data[mid_row, mid_col]

    # Format the table with standard deviation values
    stats_table = (
        f"### {data_type} Statistics\n\n"
        "| Statistic | Real Part | Imaginary Part | Magnitude | Phase/π |\n"
        "|-----------|-----------|----------------|-----------|----------|\n"
        f"| Maximum  | ${to_latex(round_number(stats['real']['max']))}$ | "
        f"${to_latex(round_number(stats['imaginary']['max']))}$ | "
        f"${to_latex(round_number(stats['magnitude']['max']))}$ | "
        f"${to_latex(round_number(stats['phase']['max']))}$ |\n"
        f"| Average  | ${to_latex(round_number(stats['real']['avg']))}$ | "
        f"${to_latex(round_number(stats['imaginary']['avg']))}$ | "
        f"${to_latex(round_number(stats['magnitude']['avg']))}$ | "
        f"${to_latex(round_number(stats['phase']['avg']))}$ |\n"
        f"| Median   | ${to_latex(round_number(stats['real']['median']))}$ | "
        f"${to_latex(round_number(stats['imaginary']['median']))}$ | "
        f"${to_latex(round_number(stats['magnitude']['median']))}$ | "
        f"${to_latex(round_number(stats['phase']['median']))}$ |\n"
        f"| Minimum  | ${to_latex(round_number(stats['real']['min']))}$ | "
        f"${to_latex(round_number(stats['imaginary']['min']))}$ | "
        f"${to_latex(round_number(stats['magnitude']['min']))}$ | "
        f"${to_latex(round_number(stats['phase']['min']))}$ |\n"
        f"| Standard Deviation | "
        f"${to_latex(round_number(stats['real']['std']))}$ | "
        f"${to_latex(round_number(stats['imaginary']['std']))}$ | "
        f"${to_latex(round_number(stats['magnitude']['std']))}$ | "
        f"${to_latex(round_number(stats['phase']['std']))}$ |\n"
        f"| Zero Point | "
        f"${to_latex(round_number(mid_value.real))}$ | "
        f"${to_latex(round_number(mid_value.imag))}$ | "
        f"${to_latex(round_number(abs(mid_value)))}$ | "
        f"${to_latex(round_number(np.angle(mid_value, deg=False) / np.pi))}$ |\n\n"
    )
    
    display_and_log_markdown(stats_table)

    # Export to Excel
    stats_df = statistics_to_dataframe(stats, data_type)
    export_statistics_to_excel(stats_df)

In [ ]:
def setup_matplotlib_style(font_size=20, label_size=22, title_size=24):
    """
    Configure the default matplotlib style for enhanced LaTeX rendering 
    and consistent font sizes in plots.
    """
    plt.style.use('default')
    plt.rcParams.update({
        'text.usetex': True,
        'font.family': 'serif',
        'font.serif': ['Computer Modern Roman'],
        'axes.formatter.use_mathtext': True,
        'font.size': font_size,
        'axes.labelsize': label_size,
        'axes.titlesize': title_size,
        'xtick.labelsize': 18,
        'ytick.labelsize': 18,
        'text.color': 'black',
        'axes.labelcolor': 'black',
        'xtick.color': 'black',
        'ytick.color': 'black',
        'text.latex.preamble': r'\usepackage{amsmath} \usepackage{amssymb}'
    })




In [ ]:
def create_colormap(scenario_key):
    """
    Select appropriate colormaps based on the given scenario.

    Args:
        scenario_key (str): Key indicating the simulation scenario.
    
    Returns:
        tuple: Four colormaps for real, imaginary, magnitude, and phase plots.
    """
    colormaps = {
        'depolarization_impact': ('RdBu', 'RdBu', 'viridis', 'twilight_shifted'),
        'theory_vs_simulation': ('seismic', 'seismic', 'viridis', 'twilight_shifted')
    }
    return colormaps.get(scenario_key, ('viridis', 'plasma', 'magma', 'twilight_shifted'))




In [ ]:
def create_subplot(fig, ax, data, extent, cmap, title, colorbar_label):
    """
    Create and format a single subplot with given data and settings.

    Args:
        fig (Figure): Matplotlib figure to hold the subplot.
        ax (Axes): Matplotlib axes to plot on.
        data (ndarray): Data to visualize in the subplot.
        extent (list): Plot extent for axes.
        cmap (str): Colormap name for the plot.
        title (str): Title for the subplot.
        colorbar_label (str): Label for the colorbar.
    
    Returns:
        AxesImage: The plotted image.
    """
    im = ax.imshow(data, extent=extent, aspect='auto', origin='lower', cmap=cmap)
    ax.set_xlabel(r'Controlled Interaction Strength $\beta/\pi$')
    ax.set_ylabel(r'Coherent Error Angle $\delta/\pi$')
    ax.set_title(title)
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label(colorbar_label)
    return im




In [ ]:
def save_figure(fig, filename, suptitle, folder='results', width=0.9):
    """
    Save a figure to a specified folder and generate LaTeX figure code.

    Args:
        fig (Figure): Matplotlib figure to save.
        filename (str): File name for the figure.
        suptitle (str): Caption for the figure in LaTeX.
        folder (str): Target folder for saving the figure.
        width (float): Width of the figure in LaTeX.
    """
    # Ensure the folder exists
    os.makedirs(folder, exist_ok=True)

    # Save the figure
    filepath = os.path.join(folder, filename)
    fig.savefig(filepath, bbox_inches='tight', dpi=300, format='png')

    # Prepare LaTeX figure code
    caption = suptitle.replace('\n', '. ')  # Format caption for LaTeX
    label = f"fig:{os.path.splitext(filename)[0]}"  # Extract name without extension
    latex_code = (
        f"\\begin{{figure}}[ht]\n"
        f"\\centering\n"
        f"\\includegraphics[width={width}\\textwidth]{{{filepath}}}\n"
        f"\\caption{{{caption}}}\n"
        f"\\label{{{label}}}\n"
        f"\\end{{figure}}"
    )

    # Display and log the LaTeX code
    display_and_log_print(latex_code)




In [ ]:
def create_standard_plots(betas, deltas, data, extent, scenario_key, scenario_title, base_subtitle,
                         cmap_real, cmap_imag, cmap_mag, cmap_phase):
    """
    Create and save the standard (non-zoomed) plots for both Cartesian and Polar coordinates.
    
    Args:
        betas, deltas: Arrays of measurement angles
        data: Complex data array to plot
        extent: Plot extents for the axes
        scenario_key: Key for identifying the scenario
        scenario_title: Title for the scenario
        base_subtitle: Base subtitle for the plots
        cmap_*: Colormaps for different components
    """
    # Detailed subtitles for each figure
    detailed_subtitle1 = f"{scenario_title} (Cartesian Coordinates)\n{base_subtitle}"
    detailed_subtitle2 = f"{scenario_title} (Polar Coordinates)\n{base_subtitle}"
    
    # Figure 1: Cartesian representation (real and imaginary parts)
    fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), dpi=300)
    
    create_subplot(fig1, ax1, data.real, extent, cmap_real, 
                   'Real Part',
                   r'$\mathrm{Re}(\hat{S})$ [bits]')
    
    create_subplot(fig1, ax2, data.imag, extent, cmap_imag,
                   'Imaginary Part',
                   r'$\mathrm{Im}(\hat{S})$ [bits]')
    
    # Set and save the Cartesian plot
    fig1.suptitle(detailed_subtitle1, fontsize=22)
    fig1.tight_layout(rect=[0, 0, 1, 1])
    save_figure(fig1, f'{scenario_key}_cartesian.png', detailed_subtitle1)
    
    # Figure 2: Polar representation (magnitude and phase)
    fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), dpi=300)
    
    create_subplot(fig2, ax1, np.abs(data), extent, cmap_mag, 
                   'Magnitude',
                   r'$|\hat{S}|$ [bits]')
    
    create_subplot(fig2, ax2, np.angle(data) / np.pi, extent, cmap_phase,
                   'Normalized Phase',
                   r'$\angle(\hat{S}) / \pi$')
    
    # Set and save the Polar plot
    fig2.suptitle(detailed_subtitle2, fontsize=22)
    fig2.tight_layout(rect=[0, 0, 1, 1])
    save_figure(fig2, f'{scenario_key}_polar.png', detailed_subtitle2)

def create_zoomed_plots(betas, deltas, data, extent, scenario_key, scenario_title, base_subtitle,
                        cmap_real, cmap_imag, cmap_mag, cmap_phase, zoom_factor):
    """
    Create and save zoomed plots for both Cartesian and Polar coordinates.
    
    Args:
        betas, deltas: Arrays of measurement angles
        data: Complex data array to plot
        extent: Plot extents for the axes
        scenario_key: Key for identifying the scenario
        scenario_title: Title for the scenario
        base_subtitle: Base subtitle for the plots
        cmap_*: Colormaps for different components
        zoom_factor: Factor to zoom in on the central region
    """
    # Calculate zoomed extents
    center_beta = (extent[0] + extent[1]) / 2
    center_delta = (extent[2] + extent[3]) / 2
    
    # Calculate zoom width and height
    zoom_width = (extent[1] - extent[0]) / zoom_factor
    zoom_height = (extent[3] - extent[2]) / zoom_factor
    
    # Calculate new extents
    zoomed_extent = [
        center_beta - zoom_width/2,
        center_beta + zoom_width/2,
        center_delta - zoom_height/2,
        center_delta + zoom_height/2
    ]
    
    # Calculate indices for slicing the data array
    beta_indices = np.logical_and(
        betas/np.pi >= zoomed_extent[0],
        betas/np.pi <= zoomed_extent[1]
    )
    delta_indices = np.logical_and(
        deltas/np.pi >= zoomed_extent[2],
        deltas/np.pi <= zoomed_extent[3]
    )
    
    # Extract the zoomed data region
    zoomed_data = data[np.ix_(delta_indices, beta_indices)]
    
    # Detailed subtitles for zoomed figures
    zoom_subtitle1 = f"{scenario_title} (Cartesian Coordinates - {zoom_factor}x Zoom)\n{base_subtitle}"
    zoom_subtitle2 = f"{scenario_title} (Polar Coordinates - {zoom_factor}x Zoom)\n{base_subtitle}"
    
    display_and_log_print(f"Generating {zoom_factor}x zoomed plots centered at β/π = {center_beta:.3f}, δ/π = {center_delta:.3f}")
    
    # Figure 3: Zoomed Cartesian representation
    fig3, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), dpi=300)
    
    create_subplot(fig3, ax1, zoomed_data.real, zoomed_extent, cmap_real, 
                   f'Real Part (Zoomed {zoom_factor}x)',
                   r'$\mathrm{Re}(\hat{S})$ [bits]')
    
    create_subplot(fig3, ax2, zoomed_data.imag, zoomed_extent, cmap_imag,
                   f'Imaginary Part (Zoomed {zoom_factor}x)',
                   r'$\mathrm{Im}(\hat{S})$ [bits]')
    
    # Set and save the zoomed Cartesian plot
    fig3.suptitle(zoom_subtitle1, fontsize=22)
    fig3.tight_layout(rect=[0, 0, 1, 1])
    save_figure(fig3, f'{scenario_key}_cartesian_zoomed_{zoom_factor}x.png', zoom_subtitle1)
    
    # Figure 4: Zoomed Polar representation
    fig4, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), dpi=300)
    
    create_subplot(fig4, ax1, np.abs(zoomed_data), zoomed_extent, cmap_mag, 
                   f'Magnitude (Zoomed {zoom_factor}x)',
                   r'$|\hat{S}|$ [bits]')
    
    create_subplot(fig4, ax2, np.angle(zoomed_data) / np.pi, zoomed_extent, cmap_phase,
                   f'Normalized Phase (Zoomed {zoom_factor}x)',
                   r'$\angle(\hat{S}) / \pi$')
    
    # Set and save the zoomed Polar plot
    fig4.suptitle(zoom_subtitle2, fontsize=22)
    fig4.tight_layout(rect=[0, 0, 1, 1])
    save_figure(fig4, f'{scenario_key}_polar_zoomed_{zoom_factor}x.png', zoom_subtitle2)

In [ ]:
def plot_results(betas, deltas, data, scenario_key, initial_state_string, zoom_factor=None):
    """
    Generate visualizations of quantum measurement results across two coordinate systems:
    1. Cartesian coordinates (real and imaginary parts)
    2. Polar coordinates (magnitude and phase)

    Args:
        betas (array): Array of measurement angles for the Ry and Rz gates.
        deltas (array): Array of coherent error angles for the Rx gate.
        data (complex ndarray): Complex array containing pseudo-entropy or related values to plot.
        scenario_key (str): Key identifying the type of data being analyzed, used to select titles and color schemes.
        initial_state_string (str): String representation of the initial quantum state.
        zoom_factor (float, optional): Factor to zoom in on central region of the plot. If None, no zoom is applied.

    Description:
        This function visualizes the results of quantum simulations for various scenarios. 
        Four figures can be generated depending on parameters:
        - The first figure illustrates the **real** and **imaginary** components of the data.
        - The second figure presents the **magnitude** and **phase** components.
        - If zoom_factor is provided, two additional zoomed figures are generated.
        Each plot uses colormaps to emphasize the structure of the data.

        Results are saved as PNG files and displayed interactively. Statistics for the data are calculated
        and presented in a tabular format.

    Returns:
        None
    """
    # Set up the global Matplotlib style for consistent aesthetics
    setup_matplotlib_style()
    
    # Define plot extents and colormaps based on input parameters and scenario type
    extent = [betas[0] / np.pi, betas[-1] / np.pi, deltas[0] / np.pi, deltas[-1] / np.pi]
    cmap_real, cmap_imag, cmap_mag, cmap_phase = create_colormap(scenario_key)
    
    # Scenario-specific titles for plots
    scenario_titles = {
        'coherent_only': "Coherent Routing Error Analysis",
        'coherent_and_depolarization': "Combined Coherent Routing Error and Depolarization Noise Evaluation",
        'depolarization_impact': "Depolarization Noise Comprehensive Impact Study",
        'theory_vs_simulation': "Pseudo-Entropy: Approximity Modeling vs Simulation Comparison"
    }
    scenario_title = scenario_titles.get(scenario_key, "Unknown Scenario")
    
    # Base subtitle shared across figures
    base_subtitle = (
        f"Simulation of Error findings with Discrete Quantum Circuits\n"
        f"Pseudo-Entropy Calculation between Initial State and Final State\n"
        f'Initial State: ${initial_state_string}$'
    )
    
    # Generate standard plots
    create_standard_plots(betas, deltas, data, extent, scenario_key, scenario_title, base_subtitle,
                         cmap_real, cmap_imag, cmap_mag, cmap_phase)
    
    # Generate zoomed plots if zoom_factor is provided
    if zoom_factor is not None:
        create_zoomed_plots(betas, deltas, data, extent, scenario_key, scenario_title, base_subtitle,
                         cmap_real, cmap_imag, cmap_mag, cmap_phase, zoom_factor)
    
    # Compute and display statistics for the data
    stats = calculate_complex_statistics(data)
    
    # Display the figures interactively
    plt.show()

# Data Analysis Functions

This section orchestrates the entire simulation and analysis pipeline. It defines the parameter grids, runs the simulations, performs the pseudo-entropy calculations, and generates all the required plots and statistical outputs. It also includes functions for fitting the pseudo-entropy data to parabolic and linear models, calculating parameter errors, and exporting results to Excel for detailed external analysis.

## Execute Experiments

This subsection manages the execution or loading of simulation results. It ensures that the `results` directory exists and handles the caching of computationally intensive pseudo-entropy calculations to avoid redundant computations.

In [ ]:
# Ensure the results directory exists
os.makedirs('results', exist_ok=True)

# Create or clear the output.txt file
with open('results/output.txt', 'w') as file:
    pass  # This creates/clears the file without writing anything

In [ ]:
# Generate angle ranges
betas = np.linspace(-BETA_RANGE, BETA_RANGE, BETA_POINTS)
deltas = np.linspace(-DELTA_RANGE, DELTA_RANGE, DELTA_POINTS)

In [ ]:
# Calculate step sizes
beta_step = (betas[-1] - betas[0]) / (BETA_POINTS - 1)
delta_step = (deltas[-1] - deltas[0]) / (DELTA_POINTS - 1)

In [ ]:
# Prepare and display range and step size information as a Markdown table
parameter_info_table = (
    "### Angle Range and Step Size Information\n\n"
    "| Parameter | Minimum ($\\pi$) | Maximum ($\\pi$) | Step Size ($\\pi$) | Number of Points |\n"
    "|-----------|-----------------|-----------------|-------------------|------------------|\n"
    f"| $\\beta$  | ${to_latex(round_number(betas[0] / np.pi))}$ | ${to_latex(round_number(betas[-1] / np.pi))}$ | ${to_latex(round_number(beta_step / np.pi))}$ | {BETA_POINTS} |\n"
    f"| $\\delta$ | ${to_latex(round_number(deltas[0] / np.pi))}$ | ${to_latex(round_number(deltas[-1] / np.pi))}$ | ${to_latex(round_number(delta_step / np.pi))}$ | {DELTA_POINTS} |\n"
)

In [ ]:
# Display the table
display_and_log_markdown(parameter_info_table)
# Print Machine epsilon
display_and_log_markdown(f"**Machine epsilon** ($\\epsilon$): ${to_latex(round_number(np.finfo(float).eps))}$")



## Run the full experiment with both numerical and analytical simulations

This section orchestrates the execution of the full experiment, running or loading simulations for both numerical and (conceptually) analytical scenarios. The `load_or_run_entropy_calculation` function manages the persistence of simulation results, ensuring that lengthy computations are only performed once.

In [ ]:
def load_or_run_entropy_calculation(file_path, betas, deltas, noise_model):
    """
    Load pseudo-entropy and std results from a file if available; otherwise, execute quantum circuits
    and calculate values, then save the results.
    
    Args:
        file_path (str): Path to the Pickle file to load/save results
        betas (array): Array of angles for Ry and Rz rotations
        deltas (array): Array of angles for additional Rx rotation (coherent error)
        noise_model (NoiseModel): Quantum noise model
    
    Returns:
        tuple: (pseudo_entropy, std_deviation) where each is a 2D array
    """
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    if os.path.exists(file_path):
        display_and_log_print(f"Loading results from {file_path}...")
        with open(file_path, "rb") as f:
            result = pickle.load(f)
    else:
        display_and_log_print(f"No file found at {file_path}. Calculating results...")
        result = run_circuits_and_calculate_entropy(betas, deltas, noise_model)
        display_and_log_print(f"Saving results to {file_path}...")
        with open(file_path, "wb") as f:
            pickle.dump(result, f)
    return result




In [ ]:
# Run or load simulations for both cases
display_and_log_print("Running numerical simulations...")
numerical_results_path = "data/numerical_results.pkl"
numerical_entropies, numerical_std_deviations, numerical_traces = load_or_run_entropy_calculation(
    numerical_results_path, betas, deltas, None
)

# Results Analysis
This section presents the core results of the pseudo-entropy simulations and their analysis. It includes displaying the quantum circuits, processing different simulation scenarios (e.g., coherent errors only), plotting pseudo-entropy components, and fitting the data to analytical models.

In [ ]:
# Display the quantum circuits used in the simulations
display_and_log_print("\n--- Step 1: Displaying the quantum circuits used in the simulations ---")
display_all_circuits()

In [ ]:
# Define scenarios for analysis and plotting
scenarios = [
    ("coherent_only", numerical_entropies, "numerical simulation with coherent noise only."),
]

### Analyzing Coherent Routing Errors
Now, we will analyze the impact of coherent routing errors on the quantum circuits. This involves quantifying the deviation of the actual circuit behavior from the ideal, numerical scenario. We'll use several metrics, including the pseudo-entropy, to characterize these errors and identify potential strategies for mitigating their effects.

In [ ]:
# Process each scenario: display statistics and plot results
for scenario_key, entropy_data, description in scenarios:
    display_and_log_print(f"\n--- Step 2: Processing Scenario: {description} ---")
    
    # Plot results
    try:
        display_and_log_print(f"Plotting: {description}")
        plot_results(betas, deltas, entropy_data, scenario_key, "|+1\\rangle")
        # Compute and display statistics for the data
        stats = calculate_complex_statistics(entropy_data)
        display_statistics_table(stats, scenario_key, entropy_data)

    except Exception as e:
        display_and_log_print(f"An error occurred while plotting {description}: {e}")

In [ ]:
def determine_best_fit(fit_results, part):
    """
    Determines the best fit (parabolic or linear) for a given part (real or imaginary) based on R-squared and p-value.

    Args:
        fit_results (dict): Dictionary containing fit results for different models.
        part (str): Part of the entropy ('real' or 'imaginary').

    Returns:
        tuple: Best fit type ('parabolic' or 'linear') and reason for the choice.
    """
    best_fit = None
    best_r2 = -np.inf
    best_p = 0
    
    for fit_type in ['parabolic', 'linear']:
        if fit_type in fit_results[part]:
            result = fit_results[part][fit_type]
            r2 = result['r2']
            p = result['p_value']
            if r2 > best_r2 or (r2 == best_r2 and p < best_p):
                best_fit = fit_type
                best_r2 = r2
                best_p = p
    
    if best_fit == 'parabolic':
        reason = "Parabolic fit is better (R² improvement: {}, p-value: {})".format(best_r2 - fit_results[part]['linear']['r2'], best_p)
    elif best_fit == 'linear':
        reason = "Linear fit preferred (similar performance, simpler model)"
    else:
        reason = "No successful fits available"
    
    return best_fit, reason

In [ ]:
def fit_entropy_vs_beta_delta(beta, delta, entropy_data):
    """
    Fits 2D parabolic and linear functions to the real and imaginary parts of the entropy data.
    Handles NaN values by filtering them out before fitting.

    Args:
        beta (array-like): Array of beta values.
        delta (array-like): Array of delta values.
        entropy_data (complex ndarray): 2D array of complex entropy values.

    Returns:
        dict: Fit results for both real and imaginary parts, including parameters, R-squared, p-value, and other metrics.

    Fits the data to the following models:
    - Parabolic: `S(beta, delta) = a * beta^2 + b * beta * delta + c * delta^2 + d * beta + e * delta + f`
    - Linear: `S(beta, delta) = m1 * beta + m2 * delta + b`
    """
    fit_results = {'real': {}, 'imaginary': {}}

    for part in ['real', 'imaginary']:
        data = entropy_data.real if part == 'real' else entropy_data.imag

        # Create meshgrid for beta and delta values
        beta_grid, delta_grid = np.meshgrid(beta, delta)
        beta_flat = beta_grid.flatten()
        delta_flat = delta_grid.flatten()
        data_flat = data.flatten()

        # Filter out NaN values
        valid_mask = ~np.isnan(data_flat)
        if np.sum(valid_mask) == 0:
            display_and_log_print(f"All data points are NaN for {part} part. Skipping fitting.")
            continue
        elif np.sum(valid_mask) < 6:  # Need at least 6 points for parabolic fit (6 parameters)
            display_and_log_print(f"Insufficient valid data points ({np.sum(valid_mask)}) for {part} part. Skipping fitting.")
            continue

        beta_valid = beta_flat[valid_mask]
        delta_valid = delta_flat[valid_mask]
        data_valid = data_flat[valid_mask]

        # Store information about filtered data
        n_total = len(data_flat)
        n_valid = len(data_valid)
        n_nan = n_total - n_valid
        
        display_and_log_print(f"Fitting {part} part: {n_valid}/{n_total} valid points ({n_nan} NaN values filtered)")

        # Fit 2D parabolic function
        try:
            def parabolic(xy, a, b, c, d, e, f):
                x, y = xy
                return a * x**2 + b * x * y + c * y**2 + d * x + e * y + f

            with warnings.catch_warnings(record=True) as w:
                popt_para, pcov_para = curve_fit(parabolic, (beta_valid, delta_valid), data_valid)
                para_warning = len(w) > 0

                y_para = parabolic((beta_valid, delta_valid), *popt_para)
                r2_para = r2_score(data_valid, y_para)

                min_val = min(np.min(data_valid), np.min(y_para))
                if min_val < 0:
                    data_shifted = data_valid - min_val + 1
                    y_para_shifted = y_para - min_val + 1
                else:
                    data_shifted = data_valid
                    y_para_shifted = y_para

                chi2_para, p_para = chisquare(data_shifted, y_para_shifted)

                fit_results[part]['parabolic'] = {
                    'params': popt_para,
                    'cov': pcov_para,
                    'r2': r2_para,
                    'chi2': chi2_para,
                    'p_value': p_para,
                    'warning': para_warning,
                    'predictions': y_para,
                    'n_valid_points': n_valid,
                    'n_nan_filtered': n_nan
                }
        except (RuntimeError, ValueError) as e:
            display_and_log_print(f"2D parabolic fitting failed for {part} part: {str(e)}")

        # Fit 2D linear function
        try:
            def linear(xy, m1, m2, b):
                x, y = xy
                return m1 * x + m2 * y + b

            # Check if we have enough points for linear fit (3 parameters)
            if n_valid < 3:
                display_and_log_print(f"Insufficient valid data points ({n_valid}) for linear fit of {part} part.")
                continue

            with warnings.catch_warnings(record=True) as w:
                popt_lin, pcov_lin = curve_fit(linear, (beta_valid, delta_valid), data_valid)
                lin_warning = len(w) > 0

                y_lin = linear((beta_valid, delta_valid), *popt_lin)
                r2_lin = r2_score(data_valid, y_lin)

                if min_val < 0:
                    y_lin_shifted = y_lin - min_val + 1
                else:
                    y_lin_shifted = y_lin

                chi2_lin, p_lin = chisquare(data_shifted, y_lin_shifted)

                fit_results[part]['linear'] = {
                    'params': popt_lin,
                    'cov': pcov_lin,
                    'r2': r2_lin,
                    'chi2': chi2_lin,
                    'p_value': p_lin,
                    'warning': lin_warning,
                    'predictions': y_lin,
                    'n_valid_points': n_valid,
                    'n_nan_filtered': n_nan
                }
        except (RuntimeError, ValueError) as e:
            display_and_log_print(f"2D linear fitting failed for {part} part: {str(e)}")

    if all(not fit_results[part] for part in ['real', 'imaginary']):
        return None

    # Determine best fit for each part (real and imaginary)
    best_fit_real, reason_real = determine_best_fit(fit_results, 'real')
    best_fit_imag, reason_imag = determine_best_fit(fit_results, 'imaginary')

    # Add the best fit and reasons to the fit_results dictionary
    fit_results['real']['best_fit'] = best_fit_real
    fit_results['real']['reason'] = reason_real
    fit_results['imaginary']['best_fit'] = best_fit_imag
    fit_results['imaginary']['reason'] = reason_imag

    return fit_results

In [ ]:
def calculate_parameter_errors(fit_results):
    """
    Calculates absolute and relative errors for the parameters of the fitted models.

    Args:
        fit_results (dict): Dictionary containing fit results for different models.

    Returns:
        dict: Dictionary containing absolute and relative errors for each parameter and fit type.
    """
    errors = {}
    for part in ['real', 'imaginary']:
        errors[part] = {}
        for fit_type in ['parabolic', 'linear']:
            if fit_type in fit_results[part]:
                result = fit_results[part][fit_type]
                if result['warning']:
                    params = result['params']
                    errors[part][fit_type] = {
                        'abs': np.full_like(params, np.nan),
                        'rel': np.full_like(params, np.nan)
                    }
                else:
                    params = result['params']
                    abs_errors = np.sqrt(np.diag(result['cov']))
                    rel_errors = np.abs(abs_errors / params * 100) if np.all(params != 0) else np.full_like(params, np.nan)
                    errors[part][fit_type] = {
                        'abs': abs_errors,
                        'rel': rel_errors
                    }
    return errors

In [ ]:
def generate_markdown_parameter_table(result, error, param_names):
    """
    Generate markdown parameter table for fit results.
    
    Args:
        result (dict): Fit result dictionary
        error (dict): Error dictionary
        param_names (list): List of parameter names
    
    Returns:
        str: Markdown formatted parameter table
    """
    markdown_output = "| Parameter | Value | Absolute Error | Relative Error (%) | Ratio to Last |\n"
    markdown_output += "|-----------|-------|----------------|--------------------|---------------|\n"
    
    # Last parameter for ratio calculations
    last_param = result['params'][-1]
    
    for i, (param, name) in enumerate(zip(result['params'], param_names)):
        abs_err = error['abs'][i]
        rel_err = error['rel'][i]
        
        # Compute ratio to the last parameter
        ratio_to_last = param / last_param if last_param != 0 else np.nan
        
        if np.isnan(abs_err):
            markdown_output += f"| ${name}$ | ${to_latex(round_number(param))}$ | undefined | uncertain | undefined |\n"
        else:
            markdown_output += (
                f"| ${name}$ | ${to_latex(round_number(param))}$ | "
                f"${to_latex(round_number(abs_err))}$ | ${to_latex(round_number(param))}$% | "
                f"${to_latex(round_number(ratio_to_last))}$ |\n"
            )
    
    return markdown_output

def generate_markdown_fit_metrics(result):
    """
    Generate markdown goodness of fit metrics.
    
    Args:
        result (dict): Fit result dictionary
    
    Returns:
        str: Markdown formatted fit metrics
    """
    markdown_output = "\n**Goodness of Fit Metrics:**\n\n"
    markdown_output += "| Metric | Value |\n"
    markdown_output += "|-----------|-------|\n"
    markdown_output += f"| $R^2$ | ${to_latex(round_number(result['r2']))}$ |\n"
    if 'chi2' in result:
        markdown_output += f"| $\\chi^2$ | ${to_latex(round_number(result['chi2']))}$ |\n"
    
    if 'p_value' in result:
        markdown_output += f"| p-value | ${to_latex(round_number(result['p_value']))}$ |\n"
    
    if result.get('warning', False):
        markdown_output += "\n*Warning: Fit parameter uncertainties may be unreliable.*\n"
    
    return markdown_output

def generate_latex_parameter_table(fit_results, errors, param_names, fit_type):
    """
    Generate LaTeX parameter table for fit results with full horizontal lines.

    Args:
        fit_results (dict): Fit results dictionary.
        errors (dict): Errors dictionary.
        param_names (list): List of parameter names.
        fit_type (str): The fit name/type to be used in caption and label.

    Returns:
        str: LaTeX formatted parameter table.
    """
    latex_output = r"\begin{table}[H]" + "\n"
    latex_output += r"\centering" + "\n"
    
    # Update caption to include fit_type
    latex_output += fr"\caption{{Parameter Values and Errors - {fit_type.title()} Fit}}\n"
    
    # Add label using fit_type
    latex_output += fr"\label{{tab:{fit_type.lower().replace(' ', '_')}}}" + "\n"

    # Add table structure with full horizontal lines
    latex_output += r"\begin{tabular}{|l|cc|cc|}" + "\n"
    latex_output += r"\hline" + "\n"
    latex_output += r"Parameter & \multicolumn{2}{c|}{Real Part} & \multicolumn{2}{c|}{Imaginary Part} \\" + "\n"
    latex_output += r"\hline" + "\n"
    latex_output += r" & Value & Abs. Error & Value & Abs. Error \\" + "\n"
    latex_output += r"\hline" + "\n"
    
    for i, name in enumerate(param_names):
        latex_output += (
            f"${name}$ & "
            f"${to_latex(round_number(fit_results['real']['params'][i]))}$ & "
            f"${to_latex(round_number(errors['real']['abs'][i]))}$ & "
            f"${to_latex(round_number(fit_results['imaginary']['params'][i]))}$ & "
            f"${to_latex(round_number(errors['imaginary']['abs'][i]))}$ \\\\\n"
        )
        latex_output += r"\hline" + "\n"

    latex_output += r"\end{tabular}" + "\n"
    latex_output += r"\end{table}" + "\n"
    
    return latex_output

def generate_latex_metrics_table(fit_results, fit_type):
    """
    Generate LaTeX goodness of fit metrics table with full horizontal lines.
    
    Args:
        fit_results (dict): Fit results dictionary.
        fit_type (str): Type of fit (will be used in caption and label).
    
    Returns:
        str: LaTeX formatted metrics table.
    """
    # Remove '' or similar suffix if exists
    clean_fit_type = fit_type.replace('_1d', '')

    latex_output = r"\begin{table}[H]" + "\n"
    latex_output += r"\centering" + "\n"
    
    # Use fit type in caption and label
    latex_output += f"\\caption{{Goodness of Fit Metrics for {clean_fit_type.title()} Fit}}" + "\n"
    latex_output += f"\\label{{tab:metrics_{clean_fit_type}}}" + "\n"
    
    # Define table structure with consistent lines
    latex_output += r"\begin{tabular}{|l|c|c|}" + "\n"
    latex_output += r"\hline" + "\n"
    latex_output += r"Metric & Real Part & Imaginary Part \\" + "\n"
    latex_output += r"\hline" + "\n"
    
    # R² metric
    latex_output += (
        f"$R^2$ & "
        f"${to_latex(round_number(fit_results['real']['r2']))}$ & "
        f"${to_latex(round_number(fit_results['imaginary']['r2']))}$ \\\\\n"
    )
    latex_output += r"\hline" + "\n"
    
    # χ² metric (if available)
    if 'chi2' in fit_results['real']:
        latex_output += (
            f"$\\chi^2$ & "
            f"${to_latex(round_number(fit_results['real']['chi2']))}$ & "
            f"${to_latex(round_number(fit_results['imaginary']['chi2']))}$ \\\\\n"
        )
        latex_output += r"\hline" + "\n"

    # p-value metric (if available)
    if 'p_value' in fit_results['real']:
        latex_output += (
            f"p-value & "
            f"${to_latex(round_number(fit_results['real']['p_value']))}$ & "
            f"${to_latex(round_number(fit_results['imaginary']['p_value']))}$ \\\\\n"
        )
        latex_output += r"\hline" + "\n"
    
    # End table
    latex_output += r"\end{tabular}" + "\n"
    latex_output += r"\end{table}" + "\n"
    
    return latex_output

def create_excel_parameter_dataframe(fit_results, errors, param_names, fit_type):
    """
    Create a pandas DataFrame for parameter values and errors.
    
    Args:
        fit_results (dict): Fit results dictionary with 'real' and 'imaginary' keys
        errors (dict): Errors dictionary with 'real' and 'imaginary' keys
        param_names (list): List of parameter names
        fit_type (str): Type of fit (parabolic, linear, etc.)
    
    Returns:
        pd.DataFrame: DataFrame with parameter information
    """
    data = []
    
    for i, param_name in enumerate(param_names):
        row = {
            'Parameter': param_name,
            'Real_Value': fit_results['real']['params'][i],
            'Real_Abs_Error': errors['real']['abs'][i],
            'Real_Rel_Error': errors['real']['rel'][i],
            'Imaginary_Value': fit_results['imaginary']['params'][i],
            'Imaginary_Abs_Error': errors['imaginary']['abs'][i],
            'Imaginary_Rel_Error': errors['imaginary']['rel'][i],
            'Fit_Type': fit_type
        }
        data.append(row)
    
    return pd.DataFrame(data)

def create_excel_metrics_dataframe(fit_results, fit_type):
    """
    Create a pandas DataFrame for goodness of fit metrics.
    
    Args:
        fit_results (dict): Fit results dictionary with 'real' and 'imaginary' keys
        fit_type (str): Type of fit (parabolic, linear, etc.)
    
    Returns:
        pd.DataFrame: DataFrame with metrics information
    """
    data = []
    
    # R² metric
    data.append({
        'Metric': 'R²',
        'Real_Value': fit_results['real']['r2'],
        'Imaginary_Value': fit_results['imaginary']['r2'],
        'Fit_Type': fit_type
    })
    
    # χ² metric (if available)
    if 'chi2' in fit_results['real']:
        data.append({
            'Metric': 'χ²',
            'Real_Value': fit_results['real']['chi2'],
            'Imaginary_Value': fit_results['imaginary']['chi2'],
            'Fit_Type': fit_type
        })
    
    # p-value metric (if available)
    if 'p_value' in fit_results['real']:
        data.append({
            'Metric': 'p-value',
            'Real_Value': fit_results['real']['p_value'],
            'Imaginary_Value': fit_results['imaginary']['p_value'],
            'Fit_Type': fit_type
        })
    
    return pd.DataFrame(data)

def create_excel_summary_dataframe(fit_results, beta_range, n_betas_used, n_betas_total, n_deltas):
    """
    Create a pandas DataFrame for fit summary information.
    
    Args:
        fit_results (dict): Fit results dictionary
        beta_range (tuple): Beta range used in fitting
        n_betas_used (int): Number of beta points used
        n_betas_total (int): Total number of beta points
        n_deltas (int): Number of delta points
    
    Returns:
        pd.DataFrame: DataFrame with summary information
    """
    data = []
    
    if beta_range is not None:
        beta0_pi = beta_range[0] / np.pi
        beta1_pi = beta_range[1] / np.pi
        
        data.append({
            'Parameter': 'Beta Range Start',
            'Value': beta0_pi,
            'Unit': 'π',
            'Description': f'Starting beta value'
        })
        
        data.append({
            'Parameter': 'Beta Range End',
            'Value': beta1_pi,
            'Unit': 'π',
            'Description': f'Ending beta value'
        })
    
    if n_betas_used is not None:
        data.append({
            'Parameter': 'Beta Points Used',
            'Value': n_betas_used,
            'Unit': 'count',
            'Description': f'Number of beta points included in fit'
        })
    
    if n_betas_total is not None:
        data.append({
            'Parameter': 'Beta Points Total',
            'Value': n_betas_total,
            'Unit': 'count',
            'Description': f'Total number of beta points available'
        })
        
        if n_betas_used is not None:
            percent = 100 * n_betas_used / n_betas_total
            data.append({
                'Parameter': 'Beta Range Coverage',
                'Value': percent,
                'Unit': '%',
                'Description': f'Percentage of beta range used'
            })
    
    if n_deltas is not None:
        data.append({
            'Parameter': 'Delta Points',
            'Value': n_deltas,
            'Unit': 'count',
            'Description': f'Number of delta points used'
        })
    
    if n_betas_used is not None and n_deltas is not None:
        n_points = n_betas_used * n_deltas
        data.append({
            'Parameter': 'Total Data Points',
            'Value': n_points,
            'Unit': 'count',
            'Description': f'Total number of data points in fit'
        })
    
    # Best fit information
    for part in ['real', 'imaginary']:
        if 'best_fit' in fit_results[part]:
            data.append({
                'Parameter': f'Best Fit ({part.title()})',
                'Value': fit_results[part]['best_fit'],
                'Unit': 'model',
                'Description': f"Best fit model for {part} part"
            })
            
            if 'reason' in fit_results[part]:
                data.append({
                    'Parameter': f'Best Fit Reason ({part.title()})',
                    'Value': fit_results[part]['reason'],
                    'Unit': 'text',
                    'Description': f"Reason for best fit selection ({part})"
                })
    
    return pd.DataFrame(data)

def export_to_excel(fit_results, errors, beta_range, n_betas_used, n_betas_total, n_deltas, 
                   fit_vs="beta"):
    """
    Export fit results to Excel file with multiple sheets.
    
    Args:
        fit_results (dict): Dictionary containing fit results for different models
        errors (dict): Dictionary containing parameter errors
        beta_range (tuple): Tuple (start_beta, end_beta) specifying the range of beta values
        n_betas_used (int): Number of beta points included in the fit
        n_betas_total (int): Total number of beta points available
        n_deltas (int): Number of delta values used
        fit_vs (str): Parameter being varied ('beta' or 'delta')
    
    Returns:
        str: Path to the saved Excel file
    """
    output_dir = "results"

    # Define parameter names based on fit dimension
    beta_symbol = "β"
    delta_symbol = "δ"
    
    param_names = {
        'parabolic': [f'{beta_symbol}²', f'{beta_symbol} {delta_symbol}', f'{delta_symbol}²', 
                     beta_symbol, delta_symbol, 'Constant'],
        'linear': [f'm₁ ({beta_symbol})', f'm₂ ({delta_symbol})', 'Constant']
    }
    
    # Generate timestamp for filename
    filename = f"fit_results.xlsx"
    filepath = os.path.join(output_dir, filename)
    
    # Create Excel writer
    with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
        
        # Summary sheet
        summary_df = create_excel_summary_dataframe(fit_results, beta_range, n_betas_used, n_betas_total, n_deltas)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        
        # Parameter sheets for each fit type
        fit_types = ['parabolic', 'linear']
        all_params_data = []
        all_metrics_data = []
        
        for fit_type in fit_types:
            if (fit_type in fit_results.get('real', {}) and 
                fit_type in fit_results.get('imaginary', {})):
                
                # Create parameter DataFrame
                params_df = create_excel_parameter_dataframe(
                    {'real': fit_results['real'][fit_type], 
                     'imaginary': fit_results['imaginary'][fit_type]},
                    {'real': errors['real'][fit_type], 
                     'imaginary': errors['imaginary'][fit_type]},
                    param_names[fit_type], fit_type
                )
                
                # Create metrics DataFrame
                metrics_df = create_excel_metrics_dataframe(
                    {'real': fit_results['real'][fit_type], 
                     'imaginary': fit_results['imaginary'][fit_type]},
                    fit_type
                )
                
                # Save individual sheets
                params_df.to_excel(writer, sheet_name=f'{fit_type.title()}_Parameters', index=False)
                metrics_df.to_excel(writer, sheet_name=f'{fit_type.title()}_Metrics', index=False)
                
                # Collect for combined sheets
                all_params_data.append(params_df)
                all_metrics_data.append(metrics_df)
        
        # Combined sheets
        if all_params_data:
            combined_params_df = pd.concat(all_params_data, ignore_index=True)
            combined_params_df.to_excel(writer, sheet_name='All_Parameters', index=False)
        
        if all_metrics_data:
            combined_metrics_df = pd.concat(all_metrics_data, ignore_index=True)
            combined_metrics_df.to_excel(writer, sheet_name='All_Metrics', index=False)
    
    return filepath

def print_fit_results(
    fit_results, errors, beta_range, n_betas_used, n_betas_total, n_deltas, fit_vs="beta"
):
    """
    Prints a formatted summary of the fit results, including parameter values, uncertainties, 
    goodness-of-fit metrics, and the actual beta range and data subset used for fitting.
    Also exports results to Excel file.

    Args:
        fit_results (dict): Dictionary containing fit results for different models.
        errors (dict): Dictionary containing parameter errors.
        beta_range (tuple): Tuple (start_beta, end_beta) specifying the range of beta values used for fitting.
        n_betas_used (int): Number of beta points included in the fit.
        n_betas_total (int): Total number of beta points available in the data.
        n_deltas (int): Number of delta values used in the fit.
        fit_vs (str, optional): Parameter being varied ('beta' or 'delta'). Default is 'beta'.

    Prints:
        - The actual beta range (in units of π) used for fitting.
        - The number of beta and delta points included, and the percentage of the beta range used.
        - Formatted tables of fit parameters, uncertainties, and goodness-of-fit metrics.
    """
    if beta_range is not None and n_betas_used is not None and n_betas_total is not None:
        beta0_pi = beta_range[0] / np.pi
        beta1_pi = beta_range[1] / np.pi
        n_points = n_betas_used * n_deltas
        percent = 100 * n_betas_used / n_betas_total
        display_and_log_markdown(f"""Fit performed over beta range: {beta0_pi:.3f}π to {beta1_pi:.3f}π 
                                 ({n_betas_used} beta points × {n_deltas} delta points = {n_points} total points, 
                                 {percent:.1f}% of beta range)""")
    elif beta_range is not None:
        beta0_pi = beta_range[0] / np.pi
        beta1_pi = beta_range[1] / np.pi
        display_and_log_markdown(f"Fit performed over beta range: {beta0_pi:.3f}π to {beta1_pi:.3f}π")

    # Export to Excel
    try:
        excel_path = export_to_excel(fit_results, errors, beta_range, n_betas_used, n_betas_total, n_deltas, fit_vs)
        display_and_log_print(f"✓ Results exported to Excel: {excel_path}")
    except Exception as e:
        display_and_log_print(f"⚠ Warning: Could not export to Excel: {e}")

    # Set the parameter symbol dynamically based on fit_vs (e.g., \beta or \δ)
    param_symbol = f"\\{fit_vs}"
    
    # Define parameter names based on fit dimension
    beta_symbol = r"\beta"
    delta_symbol = r"\delta"
    
    param_names = {
        'parabolic': [f'{beta_symbol}^2', f'{beta_symbol} {delta_symbol}', f'{delta_symbol}^2', beta_symbol, delta_symbol, 'Constant'],
        'linear': [f'm_1 ({beta_symbol})', f'm_2 ({delta_symbol})', 'Constant']
    }
    
    markdown_output = "\n### Fit Analysis of Pseudo-Entropy $\\hat{S}$: Parabolic vs. Linear Models for $\\beta$ and $\\delta$ Parameters\n"
    
    # Explanation of fit equations
    markdown_output += (
        f"\nIn this analysis, the pseudo-entropy $\\hat{{S}}$ is fitted using two models:\n\n"
        f"- **Parabolic Fit**: $\\hat{{S}}({beta_symbol}, {delta_symbol}) \\approx a {beta_symbol}^2 + b {beta_symbol} {delta_symbol} + c {delta_symbol}^2 + d {beta_symbol} + e {delta_symbol} + f$\n"
        f"- **Linear Fit**: $\\hat{{S}}({beta_symbol}, {delta_symbol}) \\approx m_1 {beta_symbol} + m_2 {delta_symbol} + b$\n\n"
    )
    
    # Prepare LaTeX output
    latex_output = ""
    
    # Set fit types
    fit_types = ['parabolic', 'linear']

    # Iterate through real and imaginary parts for markdown
    for part in ['real', 'imaginary']:
        part_symbol = r"\Re \hat{S}" if part == 'real' else r"\Im \hat{S}"
        best_fit = fit_results[part]['best_fit']
        reason = fit_results[part]['reason']

        markdown_output += f"\n#### ${part_symbol}$ Part:\n"
        markdown_output += f"**Best fit model**: *{best_fit.title()}* \n"
        markdown_output += f"**Reason**: {reason}\n\n"
            
        
        for fit_type in fit_types:
            if fit_type in fit_results[part]:
                result = fit_results[part][fit_type]
                error = errors[part][fit_type]

                markdown_output += f"\n**{fit_type.capitalize()} Fit:**\n\n"

                # Parameter Table
                markdown_output += generate_markdown_parameter_table(result, error, param_names[fit_type])

                # Goodness of Fit Metrics
                markdown_output += generate_markdown_fit_metrics(result)

    # Generate LaTeX Tables
    for fit_type in fit_types:
        latex_output += f"\\section*{{{fit_type.capitalize()} Fit}}\n"
        
        # Mention best fit
        if fit_type == fit_results['real']['best_fit']:
            latex_output += (
                "\\noindent\\textbf{Best fit for} $\\Re \\hat{S}$: "
                f"\\textit{{{fit_results['real']['best_fit'].capitalize()}}}\\\\\n"
                f"\\textbf{{Reason}}: {fit_results['real']['reason']}\\\\\n"
            )

        if fit_type == fit_results['imaginary']['best_fit']:
            latex_output += (
                "\\noindent\\textbf{Best fit for} $\\Im \\hat{S}$: "
                f"\\textit{{{fit_results['imaginary']['best_fit'].capitalize()}}}\\\\\n"
                f"\\textbf{{Reason}}: {fit_results['imaginary']['reason']}\\\\\n"
            )
        # Parameter Table
        latex_output += generate_latex_parameter_table(
            {'real': fit_results['real'][fit_type], 'imaginary': fit_results['imaginary'][fit_type]},
            {'real': errors['real'][fit_type], 'imaginary': errors['imaginary'][fit_type]},
            param_names[fit_type], fit_type
        )

        # Goodness of Fit Metrics Table
        latex_output += generate_latex_metrics_table(
            {'real': fit_results['real'][fit_type], 'imaginary': fit_results['imaginary'][fit_type]}, fit_type
        )

    # Display outputs
    display_and_log_markdown(markdown_output)
    display_and_log_print(latex_output)

# Small Angle Approximation Derivation for Pseudo-Entropy

## Introduction

This document provides a comprehensive derivation of the small angle approximation for pseudo-entropy in quantum circuits, focusing on the analytical treatment of coherent errors and weak interactions. The derivation demonstrates how the pseudo-entropy $\check{S}(\beta, \delta)$ behaves under small perturbations in the interaction strength $\beta$ and coherent error parameter $\delta$.

## Theoretical Framework

### Initial Setup

We consider a two-qubit system with initial state:
$$\ket{\psi_i} = \ket{+1} = \frac{1}{\sqrt{2}} \left( \ket{01} + \ket{11} \right)$$

The system undergoes a controlled evolution with:
- Interaction strength parameter $\beta$ (assumed small)
- Coherent error parameter $\delta$ (assumed small)

### Circuit Implementation

The quantum circuit implements the following sequence:

1. **CNOT gate application:**
   $$\text{CNOT} \ket{\psi_i} = \frac{1}{\sqrt{2}} \left( \ket{01} + \ket{10} \right)$$

2. **Parameterized unitary evolution on qubit 1:**
   $$U_{q_1}(\beta, \delta) = R_Z\left( \frac{\pi}{2} \right) R_Y(\beta) R_Z(\beta + \delta) R_X(\delta)$$

3. **Final state:**
   $$\ket{\psi_f} = \left( I \otimes U_{q_1}(\beta, \delta) \right) \text{CNOT} \ket{\psi_i}$$

## Small Angle Approximation Derivation

### Step 1: Rotation Matrix Expansions

For small angles $\theta$, the rotation matrices can be expanded using Taylor series:

$$R_X(\theta) = \begin{pmatrix} \cos(\theta/2) & -i\sin(\theta/2) \\ -i\sin(\theta/2) & \cos(\theta/2) \end{pmatrix} \approx \begin{pmatrix} 1 - \frac{\theta^2}{8} & -i\frac{\theta}{2} \\ -i\frac{\theta}{2} & 1 - \frac{\theta^2}{8} \end{pmatrix}$$

$$R_Y(\theta) = \begin{pmatrix} \cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2) \end{pmatrix} \approx \begin{pmatrix} 1 - \frac{\theta^2}{8} & -\frac{\theta}{2} \\ \frac{\theta}{2} & 1 - \frac{\theta^2}{8} \end{pmatrix}$$

$$R_Z(\theta) = \begin{pmatrix} e^{-i\theta/2} & 0 \\ 0 & e^{i\theta/2} \end{pmatrix} \approx \begin{pmatrix} 1 - i\frac{\theta}{2} & 0 \\ 0 & 1 + i\frac{\theta}{2} \end{pmatrix}$$

### Step 2: Combined Unitary Expansion

The combined unitary $U_{q_1}(\beta, \delta)$ can be expanded to second order in $\beta$ and $\delta$:

$$U_{q_1}(\beta, \delta) \approx U_0 + \beta U_1 + \delta U_2 + \beta^2 U_{11} + \beta\delta U_{12} + \delta^2 U_{22} + \mathcal{O}(\beta^3, \delta^3)$$

where:
- $U_0 = R_Z(\pi/2)$ (zeroth-order term)
- $U_1, U_2$ are first-order coefficient matrices
- $U_{11}, U_{12}, U_{22}$ are second-order coefficient matrices

### Step 3: State Evolution Under Small Perturbations

The final state can be written as:
$$\ket{\psi_f} = \ket{\psi_0} + \beta \ket{\psi_1} + \delta \ket{\psi_2} + \beta^2 \ket{\psi_{11}} + \beta\delta \ket{\psi_{12}} + \delta^2 \ket{\psi_{22}} + \mathcal{O}(\beta^3, \delta^3)$$

where $\ket{\psi_0}$ is the unperturbed final state and $\ket{\psi_1}, \ket{\psi_2}, \ldots$ are the perturbation terms.

### Step 4: Overlap Calculation

The overlap $\langle\psi_f|\psi_i\rangle$ expands as:
$$\langle\psi_f|\psi_i\rangle = \alpha_0 + \beta \alpha_1 + \delta \alpha_2 + \beta^2 \alpha_{11} + \beta\delta \alpha_{12} + \delta^2 \alpha_{22} + \mathcal{O}(\beta^3, \delta^3)$$

### Step 5: Generalized Density Matrix

The generalized transition matrix is:
$$\hat{\rho} = \frac{\ket{\psi_i}\bra{\psi_f}}{\langle\psi_f|\psi_i\rangle}$$

Expanding the denominator using the geometric series:
$$\frac{1}{\langle\psi_f|\psi_i\rangle} = \frac{1}{\alpha_0} \left(1 - \frac{\beta \alpha_1 + \delta \alpha_2}{\alpha_0} + \frac{(\beta \alpha_1 + \delta \alpha_2)^2}{\alpha_0^2} + \ldots\right)$$

### Step 6: Reduced Density Matrix

The reduced density matrix for qubit 0 is:
$$\hat{\rho}_{q_0} = \text{Tr}_{q_1}(\hat{\rho}) = \rho_0 + \beta \rho_1 + \delta \rho_2 + \beta^2 \rho_{11} + \beta\delta \rho_{12} + \delta^2 \rho_{22} + \mathcal{O}(\beta^3, \delta^3)$$

### Step 7: Eigenvalue Expansion

The eigenvalues $\lambda_k$ of $\hat{\rho}_{q_0}$ can be expanded as:
$$\lambda_k = \lambda_k^{(0)} + \beta \lambda_k^{(1)} + \delta \lambda_k^{(2)} + \beta^2 \lambda_k^{(11)} + \beta\delta \lambda_k^{(12)} + \delta^2 \lambda_k^{(22)} + \mathcal{O}(\beta^3, \delta^3)$$

### Step 8: Pseudo-Entropy Expansion

The pseudo-entropy is:
$$\check{S}(\beta, \delta) = -\sum_k \lambda_k \log_2 \lambda_k$$

Using the Taylor expansion of the logarithm:
$$\log_2 \lambda_k = \log_2 \lambda_k^{(0)} + \frac{1}{\lambda_k^{(0)} \ln 2}\left(\beta \lambda_k^{(1)} + \delta \lambda_k^{(2)}\right) + \frac{1}{2\lambda_k^{(0)} \ln 2}\left(\beta^2 \lambda_k^{(11)} + \beta\delta \lambda_k^{(12)} + \delta^2 \lambda_k^{(22)}\right) + \mathcal{O}(\beta^3, \delta^3)$$

## Final Result: Parabolic Approximation

Combining all terms up to second order, the pseudo-entropy takes the form:

$$\check{S}(\beta, \delta) \approx a \beta^2 + b \beta \delta + c \delta^2 + d \beta + e \delta + f$$

where the coefficients are:
- $a = -\sum_k \left[\lambda_k^{(11)} \log_2 \lambda_k^{(0)} + \frac{(\lambda_k^{(1)})^2}{\lambda_k^{(0)} \ln 2}\right]$
- $b = -\sum_k \left[\lambda_k^{(12)} \log_2 \lambda_k^{(0)} + \frac{2\lambda_k^{(1)}\lambda_k^{(2)}}{\lambda_k^{(0)} \ln 2}\right]$
- $c = -\sum_k \left[\lambda_k^{(22)} \log_2 \lambda_k^{(0)} + \frac{(\lambda_k^{(2)})^2}{\lambda_k^{(0)} \ln 2}\right]$
- $d = -\sum_k \left[\lambda_k^{(1)} \log_2 \lambda_k^{(0)} + \frac{\lambda_k^{(1)}}{\ln 2}\right]$
- $e = -\sum_k \left[\lambda_k^{(2)} \log_2 \lambda_k^{(0)} + \frac{\lambda_k^{(2)}}{\ln 2}\right]$
- $f = -\sum_k \lambda_k^{(0)} \log_2 \lambda_k^{(0)}$

## Linear Approximation

In the regime where quadratic terms are negligible, the pseudo-entropy reduces to:

$$\check{S}(\beta, \delta) \approx m_1 \beta + m_2 \delta + b$$

where:
- $m_1 = d$ (coefficient of $\beta$)
- $m_2 = e$ (coefficient of $\delta$)
- $b = f$ (constant term)

## Special Cases

### Case 1: $\beta = 0$ (No Interaction)

When $\beta = 0$, the pseudo-entropy becomes:
$$\check{S}(0, \delta) = c \delta^2 + e \delta + f$$

Analytical calculation shows that $\text{Im}[\check{S}(0, \delta)] = 0$ for all $\delta$, confirming that coherent errors alone do not introduce imaginary components in the absence of interactions.

### Case 2: $\delta = 0$ (No Coherent Error)

When $\delta = 0$, the pseudo-entropy becomes:
$$\check{S}(\beta, 0) = a \beta^2 + d \beta + f$$

This represents the pure interaction effect without coherent errors.

### Case 3: Near-Orthogonal States ($\beta \approx \pi/2$, $\delta \approx 0$)

In this regime, $\langle\psi_f|\psi_i\rangle \approx 0$, leading to:
$$\check{S}(\pi/2, \delta) \sim \frac{1}{\delta} + \mathcal{O}(\delta^0)$$

This singular behavior is handled numerically as NaN when $\delta = 0$ exactly.

## Validity and Limitations

The small angle approximation is valid when:
1. $|\beta| \ll 1$ and $|\delta| \ll 1$
2. The system remains in the perturbative regime
3. Higher-order terms $(>\mathcal{O}(\beta^2, \delta^2))$ are negligible

The approximation breaks down near:
- Orthogonal state configurations
- Large parameter values
- Regions where the overlap $\langle\psi_f|\psi_i\rangle$ approaches zero

## Conclusion

The small angle approximation provides a powerful analytical framework for understanding pseudo-entropy behavior in quantum circuits with weak interactions and coherent errors. The derived parabolic form captures the essential physics while remaining computationally tractable for fitting and analysis purposes.

In [ ]:
def compute_fitted_entropy(fit_results, beta, delta):
    """
    Computes the fitted entropy values using the best fit models for real and imaginary parts.
    
    Args:
        fit_results (dict): Dictionary containing fit results from fit_entropy_vs_beta_delta.
        beta (array-like): Array of beta values.
        delta (array-like): Array of delta values.
        
    Returns:
        complex ndarray: 2D array of fitted complex entropy values.
    """
    # Create meshgrid for beta and delta values
    beta_grid, delta_grid = np.meshgrid(beta, delta)
    beta_flat = beta_grid.flatten()
    delta_flat = delta_grid.flatten()
    
    fitted_values = np.zeros((len(delta), len(beta)), dtype=complex)
    
    for part in ['real', 'imaginary']:
        # Get the best fit type for this part
        best_fit = fit_results[part]['best_fit']
        if best_fit is None:
            continue
            
        # Get the parameters for the best fit
        params = fit_results[part][best_fit]['params']
        
        # Compute fitted values based on the model type
        if best_fit == 'parabolic':
            # S(beta, delta) = a * beta^2 + b * beta * delta + c * delta^2 + d * beta + e * delta + f
            a, b, c, d, e, f = params
            values = (a * beta_grid**2 + 
                     b * beta_grid * delta_grid + 
                     c * delta_grid**2 + 
                     d * beta_grid + 
                     e * delta_grid + 
                     f)
        else:  # linear
            # S(beta, delta) = m1 * beta + m2 * delta + b
            m1, m2, b = params
            values = m1 * beta_grid + m2 * delta_grid + b
        
        # Add to the complex array
        if part == 'real':
            fitted_values += values
        else:
            fitted_values += 1j * values
    
    return fitted_values

### Consistency Analysis and Model Performance Assessment

This section performs a detailed consistency analysis and assesses the performance of the theoretical model against simulation results. This includes fitting the pseudo-entropy data to parabolic and linear models and evaluating the goodness-of-fit. The detailed sub-subsections covering fitted parameters, consistency verification, model performance comparison, parameter hierarchy, statistical validity assessment, and visual validation are offloaded to the supplementary materials for in-depth review.

In [ ]:
def analyze_pseudo_entropy_data(betas, deltas, entropy_data):
    """
    Analyzes 2D pseudo entropy data by fitting parabolic and linear models to the real and imaginary parts,
    calculates statistical metrics for the residuals, and plots the residuals.
    Uses only a chosen central window of beta values for analysis.

    Args:
        betas (array-like): Array of beta values.
        deltas (array-like): Array of delta values.
        entropy_data (complex ndarray): 2D array of complex pseudo entropy values.

    Returns:
        None
    """
    n_betas = len(betas)
    n_deltas = len(deltas)
    # Use a narrow (empirically chosen) central window for high-quality fits
    start_idx = 7 * n_betas // 16
    end_idx = 9 * n_betas // 16

    betas_subset = betas[start_idx:end_idx]
    entropy_data_subset = entropy_data[:, start_idx:end_idx]

    print(f"Using beta range: {betas_subset[0]:.6g} to {betas_subset[-1]:.6g}")
    print(f"Total betas: {n_betas}, Using: {len(betas_subset)} ({len(betas_subset)/n_betas*100:.1f}%)")

    # Fit and analyze using the subset
    fit_results = load_or_calculate(
        "data/fit_results.pkl",
        fit_entropy_vs_beta_delta,
        betas_subset, deltas, entropy_data_subset
    )

    errors = load_or_calculate(
        "data/parameter_errors.pkl",
        calculate_parameter_errors,
        fit_results
    )

    # Pass the actual beta range used to print_fit_results
    print_fit_results(
        fit_results, errors,
        beta_range=(betas_subset[0], betas_subset[-1]),
        n_betas_used=len(betas_subset), n_betas_total=n_betas,
        n_deltas=n_deltas
    )

    # Compute fitted values using the best models
    fitted_entropy = load_or_calculate(
        "data/fitted_entropy.pkl",
        compute_fitted_entropy,
        fit_results, betas_subset, deltas
    )

    # Plot the residuals using the subset
    plot_results(betas_subset, deltas, entropy_data_subset - fitted_entropy, 'theory_vs_simulation', "|+1\\rangle")

In [ ]:
# Analyzes 2D pseudo entropy data
analyze_pseudo_entropy_data(betas, deltas, numerical_entropies)

In [ ]:
def load_or_calculate(file_path, calculate_function, *args, **kwargs):
    """
    Load a value from a file if it exists; otherwise, calculate it, save it, and return the value.

    Args:
        file_path (str): Path to the file to load/save the value.
        calculate_function (callable): Function to calculate the value if not loaded.
        *args: Positional arguments for the calculate_function.
        **kwargs: Keyword arguments for the calculate_function.

    Returns:
        Any: The loaded or calculated value.
    """
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    if os.path.exists(file_path):
        display_and_log_print(f"Loading from {file_path}...")
        with open(file_path, "rb") as f:
            value = pickle.load(f)
    else:
        display_and_log_print(f"Calculating and saving to {file_path}...")
        value = calculate_function(*args, **kwargs)
        with open(file_path, "wb") as f:
            pickle.dump(value, f)
    return value



### Focusing on Key Regions of Continuous Classical-like Behaviour

This section delves into the analysis of continuous "classical-like" segments within the pseudo-entropy parameter space. It identifies and visualizes these segments, which are crucial for understanding the robustness and sensitivity of the pseudo-entropy method to coherent errors. The analysis includes calculating Clopper-Pearson confidence intervals for the detection rates within these segments, providing a statistically rigorous assessment of the method's performance.

While this notebook provides summary metrics and visualizations, the *detailed tables and statistics per beta/delta* are saved in results directory and can be explored further. The analysis is designed to be flexible, allowing for both direct threshold values and exponent-based thresholds, enabling a comprehensive exploration of the parameter space.

In [ ]:
def analyze_segments(
    classical_mask,
    axis_values1,
    axis_values2,
    white_percentage_threshold=0.3,
    is_vertical=True,
    confidence_level=0.95
):
    """
    Identifies continuous segments in either vertical or horizontal direction.
    Calculates confidence intervals for the proportion of True values within each segment,
    ignoring np.nan values and reporting the percentage of NaNs per segment in the output.

    Parameters:
        classical_mask (ndarray): 2D boolean array (with possible np.nan for undefined values)
                                  axis_values1 corresponds to rows, axis_values2 to columns.
        axis_values1 (array): Values for the first axis (typically betas) - corresponds to rows.
        axis_values2 (array): Values for the second axis (typically deltas) - corresponds to columns.
        white_percentage_threshold (float): Minimum fraction of True values (excluding NaNs) to qualify a segment.
        is_vertical (bool): 
            If True, finds vertical segments (constant axis_values2, i.e., constant beta, varying delta).
            If False, finds horizontal segments (constant axis_values1, i.e., constant delta, varying beta).
        confidence_level (float): Confidence level for binomial confidence intervals.

    Returns:
        segment_info (list of dict): Each dictionary contains information about a continuous qualified segment:
            - 'start_pi': Start value of the segment in units of π.
            - 'end_pi': End value of the segment in units of π.
            - 'average_thickness_pi': Average thickness of the segment in units of π.
            - 'average_true_percentage': Average percentage of True (classical-like) values in the segment (excluding NaNs).
            - 'average_true_percentage_ci_lower': Lower confidence interval bound (in percent).
            - 'average_true_percentage_ci_upper': Upper confidence interval bound (in percent).
            - 'length_cells': Number of axis points in the segment.
            - 'nan_percentage': Percentage of NaN (undefined) entries within the segment.
    """
    if is_vertical:
        # Vertical segments: analyze columns (constant beta, varying delta)
        mask_to_analyze = classical_mask  # rows=delta, cols=beta
        num_segments = classical_mask.shape[1]  # number of beta values
        segment_axis_values = axis_values1  # beta values
    else:
        # Horizontal segments: analyze rows (constant delta, varying beta)
        mask_to_analyze = classical_mask.T  # transpose so rows=beta, cols=delta
        num_segments = classical_mask.shape[0]  # number of delta values
        segment_axis_values = axis_values2  # delta values

    qualified_indices = []

    for j in range(num_segments):
        line = mask_to_analyze[:, j]
        valid = ~np.isnan(line)
        true_count = np.sum(line[valid])
        segment_length = np.sum(valid)
        nan_count = np.sum(np.isnan(line))
        nan_percentage = (nan_count / len(line) * 100) if len(line) > 0 else 0
        white_percentage = (true_count / segment_length) if segment_length > 0 else 0
        if white_percentage >= white_percentage_threshold:
            qualified_indices.append(j)

    segment_info = []
    if qualified_indices:
        start_index = qualified_indices[0]
        for i in range(1, len(qualified_indices)):
            if qualified_indices[i] > qualified_indices[i - 1] + 1:
                # End of a continuous segment
                end_index = qualified_indices[i - 1]
                segment_indices = range(start_index, end_index + 1)
                segment_submask = mask_to_analyze[:, segment_indices]
                valid = ~np.isnan(segment_submask)
                total_cells_in_segment = np.sum(valid)
                true_count_in_segment = np.sum(segment_submask[valid])
                nan_count_segment = np.sum(np.isnan(segment_submask))
                nan_percentage_segment = (nan_count_segment / segment_submask.size * 100) if segment_submask.size > 0 else 0
                avg_percentage = (true_count_in_segment / total_cells_in_segment) * 100 if total_cells_in_segment > 0 else 0
                ci_lower, ci_upper = calculate_binomial_ci(true_count_in_segment, total_cells_in_segment, confidence_level)
                if len(segment_axis_values) > 1:
                    average_thickness_pi = np.abs(segment_axis_values[1] - segment_axis_values[0]) / np.pi
                else:
                    average_thickness_pi = 0
                start_value = segment_axis_values[start_index] / np.pi
                end_value = segment_axis_values[end_index] / np.pi
                segment_info.append({
                    'start_pi': start_value,
                    'end_pi': end_value,
                    'average_thickness_pi': average_thickness_pi,
                    'average_true_percentage': avg_percentage,
                    'average_true_percentage_ci_lower': ci_lower * 100,
                    'average_true_percentage_ci_upper': ci_upper * 100,
                    'length_cells': len(segment_indices),
                    'nan_percentage': nan_percentage_segment
                })
                start_index = qualified_indices[i]

        # Handle the last segment
        end_index = qualified_indices[-1]
        segment_indices = range(start_index, end_index + 1)
        segment_submask = mask_to_analyze[:, segment_indices]
        valid = ~np.isnan(segment_submask)
        total_cells_in_segment = np.sum(valid)
        true_count_in_segment = np.sum(segment_submask[valid])
        nan_count_segment = np.sum(np.isnan(segment_submask))
        nan_percentage_segment = (nan_count_segment / segment_submask.size * 100) if segment_submask.size > 0 else 0
        avg_percentage = (true_count_in_segment / total_cells_in_segment) * 100 if total_cells_in_segment > 0 else 0
        ci_lower, ci_upper = calculate_binomial_ci(true_count_in_segment, total_cells_in_segment, confidence_level)
        if len(segment_axis_values) > 1:
            average_thickness_pi = np.abs(segment_axis_values[1] - segment_axis_values[0]) / np.pi
        else:
            average_thickness_pi = 0
        start_value = segment_axis_values[start_index] / np.pi
        end_value = segment_axis_values[end_index] / np.pi
        segment_info.append({
            'start_pi': start_value,
            'end_pi': end_value,
            'average_thickness_pi': average_thickness_pi,
            'average_true_percentage': avg_percentage,
            'average_true_percentage_ci_lower': ci_lower * 100,
            'average_true_percentage_ci_upper': ci_upper * 100,
            'length_cells': len(segment_indices),
            'nan_percentage': nan_percentage_segment
        })

    return segment_info


In [ ]:
def calculate_binomial_ci(k, n, confidence_level=0.95):
    """
    Calculates the Clopper-Pearson confidence interval for a binomial proportion.
    This method is robust for proportions near 0 or 1, and for small N.

    Args:
        k (int): Number of successes (True values).
        n (int): Total number of trials (total cells).
        confidence_level (float): Desired confidence level (e.g., 0.95 for 95%).

    Returns:
        tuple: (lower_bound, upper_bound) of the confidence interval (as proportions).
               Returns (np.nan, np.nan) if n is 0 or k is invalid.
    """
    if n == 0 or not (0 <= k <= n):
        return np.nan, np.nan
    alpha = 1 - confidence_level
    
    # Clopper-Pearson interval using scipy.stats.beta.ppf
    # This approach is generally recommended for its good coverage properties.
    lower = beta.ppf(alpha / 2, k, n - k + 1)
    upper = beta.ppf(1 - alpha / 2, k + 1, n - k)

    # Ensure bounds are within [0, 1] due to potential numerical precision at extremes
    return max(0.0, lower), min(1.0, upper)




In [ ]:
def adjust_segments_for_zoom(segments, original_coords, zoomed_coords, zoomed_mask, zoomed_betas, zoomed_deltas, is_vertical):
    """
    Adjusts segment coordinates for zoomed view and recalculates percentages.

    Args:
        segments (list): List of segment dictionaries containing 'start_pi' and 'end_pi' coordinates.
        original_coords (ndarray): Original coordinate array (betas or deltas).
        zoomed_coords (ndarray): Zoomed coordinate array (subset of original).
        zoomed_mask (ndarray): The zoomed classical mask.
        zoomed_betas (ndarray): Zoomed beta coordinates.
        zoomed_deltas (ndarray): Zoomed delta coordinates.
        is_vertical (bool): True if adjusting vertical segments, False for horizontal.

    Returns:
        list: Adjusted segments with updated coordinates and recalculated percentages.
    """
    # Check if arrays are empty
    if len(original_coords) == 0 or len(zoomed_coords) == 0:
        return []
    o_start, o_end = original_coords[0], original_coords[-1]
    z_start, z_end = zoomed_coords[0], zoomed_coords[-1]
    scale = (z_end - z_start) / (o_end - o_start) if (o_end - o_start) != 0 else 1
    offset = z_start - o_start * scale
    adjusted_segments = []
    for seg_info in segments:
        new_start_pi = seg_info['start_pi'] * scale + offset
        new_end_pi = seg_info['end_pi'] * scale + offset
        # Recalculate average percentage based on the zoomed mask
        if is_vertical:
            # For vertical segments
            start_idx = np.argmin(np.abs(zoomed_betas - new_start_pi * np.pi))
            end_idx = np.argmin(np.abs(zoomed_betas - new_end_pi * np.pi))
            if start_idx > end_idx:
                start_idx, end_idx = end_idx, start_idx
            segment_mask = zoomed_mask[:, start_idx:end_idx+1]
        else:
            # For horizontal segments
            start_idx = np.argmin(np.abs(zoomed_deltas - new_start_pi * np.pi))
            end_idx = np.argmin(np.abs(zoomed_deltas - new_end_pi * np.pi))
            if start_idx > end_idx:
                start_idx, end_idx = end_idx, start_idx
            segment_mask = zoomed_mask[start_idx:end_idx+1, :]
        true_count = np.sum(segment_mask)
        total_count = segment_mask.size
        avg_percentage = (true_count / total_count) * 100 if total_count > 0 else 0
        adjusted_segments.append({**seg_info, 'start_pi': new_start_pi, 'end_pi': new_end_pi, 'average_true_percentage': avg_percentage})
    return adjusted_segments


def _create_continuous_segments_plot(classical_mask, betas, deltas, vertical_segments, horizontal_segments, imag_threshold, is_imag_threshold_exponent, zoom_factor=None, filename_suffix=""):
    """
    Helper function to create and save a visualization of continuous classical-like segments.

    Args:
        classical_mask (ndarray): Boolean mask from pseudo-entropy thresholding.
        betas (array): Values for β angles.
        deltas (array): Values for δ angles.
        vertical_segments (list): Vertical continuous segment info.
        horizontal_segments (list): Horizontal continuous segment info.
        imag_threshold (int or float): The imaginary threshold. If is_imag_threshold_exponent is True,
                                      it's an exponent (int), otherwise, it's the direct threshold value (float).
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold),
                                           if False, treat it as the direct threshold value.
        zoom_factor (float, optional): Factor to zoom in on central region of the plot. If None, no zoom is applied.
        filename_suffix (str, optional): Additional text to append to the filename. Defaults to "".
    """
    fig, ax = plt.subplots(figsize=(12, 10))

    if is_imag_threshold_exponent:
        threshold_label = f"{np.abs(imag_threshold):.0f}"
        title_threshold_str = f"10^{{{imag_threshold}}}"
        filename_threshold_str = f"{np.abs(imag_threshold)}"
    else:
        threshold_label = f"{imag_threshold*100:.2f}\\%"
        title_threshold_str = f"{threshold_label}"
        filename_threshold_str = threshold_label.replace(".", "_").replace("-", "").replace("\\%", "")


    if zoom_factor is not None and zoom_factor > 1:
        center_delta_index = len(deltas) // 2
        center_beta_index = len(betas) // 2

        delta_half_range = int((len(deltas) // (2 * zoom_factor)))
        beta_half_range = int((len(betas) // (2 * zoom_factor)))

        delta_start = max(0, center_delta_index - delta_half_range)
        delta_end = min(len(deltas), center_delta_index + delta_half_range)
        beta_start = max(0, center_beta_index - beta_half_range)
        beta_end = min(len(betas), center_beta_index + beta_half_range) # Corrected line

        zoomed_classical_mask = classical_mask[delta_start:delta_end,
                                              beta_start:beta_end]
        zoomed_betas = betas[beta_start:beta_end]
        zoomed_deltas = deltas[delta_start:delta_end]
        extent = [zoomed_betas[0] / np.pi, zoomed_betas[-1] / np.pi,
                  zoomed_deltas[0] / np.pi, zoomed_deltas[-1] / np.pi]
        im = ax.imshow(zoomed_classical_mask, cmap='gray', extent=extent,
                      origin='lower', aspect='auto')

        zoomed_vertical_segments = adjust_segments_for_zoom(
            vertical_segments, deltas, zoomed_deltas,
            zoomed_classical_mask, zoomed_betas, zoomed_deltas, True
        )

        zoomed_horizontal_segments = adjust_segments_for_zoom(
            horizontal_segments, betas, zoomed_betas,
            zoomed_classical_mask, zoomed_betas, zoomed_deltas, False
        )

        all_zoomed_segments = zoomed_vertical_segments + zoomed_horizontal_segments
        percentages = [info['average_true_percentage']
                       for info in all_zoomed_segments]
        norm = plt.Normalize(min(percentages) if percentages else 0,
                             max(percentages) if percentages else 1)
        v_cmap = plt.cm.Blues
        h_cmap = plt.cm.Greens

        for info in zoomed_vertical_segments:
            color = v_cmap(norm(info['average_true_percentage']))
            ax.axvspan(info['start_pi'], info['end_pi'], color=color, alpha=0.5)

        for info in zoomed_horizontal_segments:
            color = h_cmap(norm(info['average_true_percentage']))
            ax.axhspan(info['start_pi'], info['end_pi'], color=color, alpha=0.5)

    else:
        extent = [betas[0] / np.pi, betas[-1] / np.pi, deltas[0] / np.pi,
                  deltas[-1] / np.pi]
        im = ax.imshow(classical_mask, cmap='gray', extent=extent,
                      origin='lower', aspect='auto')
        all_segments = vertical_segments + horizontal_segments
        percentages = [info['average_true_percentage']
                       for info in all_segments]
        norm = plt.Normalize(min(percentages) if percentages else 0,
                             max(percentages) if percentages else 1)
        v_cmap = plt.cm.Blues
        h_cmap = plt.cm.Greens

        for info in vertical_segments:
            color = v_cmap(norm(info['average_true_percentage']))
            ax.axvspan(info['start_pi'], info['end_pi'], color=color, alpha=0.5)

        for info in horizontal_segments:
            color = h_cmap(norm(info['average_true_percentage']))
            ax.axhspan(info['start_pi'], info['end_pi'], color=color, alpha=0.5)

    ax.set_xlabel(r'Controlled Interaction Strength $\beta / \pi$')
    ax.set_ylabel(r'Coherent Error Angle $\delta / \pi$')
    title_suffix = f" (Zoomed-in Center {zoom_factor}x)" if zoom_factor is not None else ""
    ax.set_title(
        f'Continuous Classical-like Regions ($|\\Im(\\check{{S}})| < {title_threshold_str}$){title_suffix}',
        fontsize=16
    )

    v_sm = plt.cm.ScalarMappable(cmap=v_cmap, norm=norm)
    h_sm = plt.cm.ScalarMappable(cmap=h_cmap, norm=norm)

    plt.colorbar(v_sm, ax=ax, location='left', pad=0.1,
                         label='Vertical Segments Average True Percentage (\\%)')
    plt.colorbar(h_sm, ax=ax, location='right', pad=0.1,
                         label='Horizontal Segments Average True Percentage (\\%)')

    plt.tight_layout()
    filename = f'results/continuous_classical_regions_thresh_{filename_threshold_str}{filename_suffix}.png'
    plt.savefig(filename, bbox_inches='tight')
    plt.close(fig)

    latex_import_figure = (
        "\\begin{figure}[ht]\n"
        "    \\centering\n"
        f"    \\includegraphics[width=0.8\\textwidth]{{{filename}}}\n"
        f"    \\caption{{Continuous Classical-like Regions (Colored by Direction and Average True Percentage) "
    )
    latex_import_figure += f"with $|\\Im(\\check{{S}})| < {title_threshold_str}${title_suffix}}}\n"

    latex_import_figure += f"    \\label{{fig:continuous_classical_regions_{filename_threshold_str}{filename_suffix.replace('-', '_')}}}\n"
    latex_import_figure += "    \\end{figure}"

    display_and_log_print(latex_import_figure)

def visualize_segments(classical_mask, betas, deltas, vertical_segments,
                       horizontal_segments, imag_threshold, is_imag_threshold_exponent, zoom_factor=None):
    """
    Visualizes the classical mask and highlights continuous segments with colors based on their average true percentage.
    Optionally, creates additional zoomed-in views of the central region of the plot.

    Args:
        classical_mask (ndarray): Boolean mask from pseudo-entropy thresholding (True for classical-like).
        betas (array): Values for β angles (x-axis).
        deltas (array): Values for δ angles (y-axis).
        vertical_segments (list): List of dictionaries, each containing information about a continuous vertical segment,
                                  including 'start_pi', 'end_pi', and 'average_true_percentage'.
        horizontal_segments (list): List of dictionaries, each containing information about a continuous horizontal segment,
                                    including 'start_pi', 'end_pi', and 'average_true_percentage'.
        imag_threshold (int or float): The imaginary threshold. If is_imag_threshold_exponent is True,
                                      it's an exponent (int), otherwise, it's the direct threshold value (float).
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold),
                                           if False, treat it as the direct threshold value.
        zoom_factor (float, optional): Factor to zoom in on central region of the plot. If None, no zoom is applied.

    Description:
        This function displays the classical_mask as a grayscale image and overlays colored spans
        to highlight continuous segments of classical-like behavior. Vertical segments are colored
        using a blue colormap, and horizontal segments are colored using a green colormap. The
        intensity of the color indicates the average_true_percentage of the classical-like
        region within that segment.

        If zoom_factor is provided, an additional plot focusing on the central portion of the
        parameter space is generated, allowing for a closer inspection of specific regions and their
        identified continuous segments. The zoomed plot is saved with a "_zoom_[zoom_factor]" suffix
        in the filename.
    """
    _create_continuous_segments_plot(classical_mask, betas, deltas,
                                     vertical_segments, horizontal_segments,
                                     imag_threshold, is_imag_threshold_exponent)
    if zoom_factor is not None and zoom_factor > 1:  # Changed condition to zoom_factor > 1
        _create_continuous_segments_plot(classical_mask, betas, deltas,
                                         vertical_segments, horizontal_segments,
                                         imag_threshold, is_imag_threshold_exponent, zoom_factor,
                                         f"_zoom_{zoom_factor}")





In [ ]:
def create_segment_table(segments, direction, title_threshold_str):
    """
    Create a markdown table for either vertical or horizontal segments, including confidence intervals and NaN percentage.
    
    Args:
        segments (list): List of segment dictionaries containing segment info.
        direction (str): Direction type ("vertical" or "horizontal").
        title_threshold_str (str): Formatted threshold string for the title.
        
    Returns:
        str: Formatted markdown table as a string.
    """
    if not segments:
        return f"No continuous {direction} white lines found for $|\\Im(\\check{{S}})| < {title_threshold_str}$.\n\n"

    table = (
        f"**{direction.capitalize()} Segments (Threshold $|\\Im(\\check{{S}})| < "
        f"{title_threshold_str}$):**\n\n"
    )

    table += (
        r"| Value/π Start | Value/π End | Avg. Thickness (Value/π) | Length (Cells) | Avg. True (%) | 95\% CI Lower (%) | 95\% CI Upper (%) | NaN (%) |" + "\n"
        "| :------------: | :-----------: | :-----------------------: | :------------: | :------------: | :---------------: | :---------------: | :-----: |\n"
    )
    
    for info in segments:
        table += (
            f"| {to_latex(round_number(info['start_pi']))} "
            f"| {to_latex(round_number(info['end_pi']))} "
            f"| ${to_latex(round_number(info['average_thickness_pi']))}$ "
            f"| {info['length_cells']:>14d} "
            f"| {info['average_true_percentage']:>13.2f} "
            f"| {info['average_true_percentage_ci_lower']:>17.2f} "
            f"| {info['average_true_percentage_ci_upper']:>17.2f} "
            f"| {info['nan_percentage']:>7.2f} |\n"
        )
    
    return table + "\n"



In [ ]:
def segments_to_dataframe(segments, threshold_value):
    """
    Convert segment data to a pandas DataFrame.
    
    Args:
        segments (list): List of segment dictionaries containing segment info.
        threshold_value (float): The threshold value to include in the DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame containing segment data with proper column names.
    """
    if not segments:
        return pd.DataFrame()
    
    # Convert segments to DataFrame
    df_data = []
    for segment in segments:
        row = {
            'Threshold': threshold_value,
            'Value/π Start': segment['start_pi'],
            'Value/π End': segment['end_pi'],
            'Avg. Thickness (Value/π)': segment['average_thickness_pi'],
            'Length (Cells)': segment['length_cells'],
            'Avg. True (%)': segment['average_true_percentage'],
            '95% CI Lower (%)': segment['average_true_percentage_ci_lower'],
            '95% CI Upper (%)': segment['average_true_percentage_ci_upper'],
            'NaN (%)': segment['nan_percentage']
        }
        df_data.append(row)
    
    return pd.DataFrame(df_data)


def write_segments_to_excel(vertical_segments, horizontal_segments, imag_threshold, is_imag_threshold_exponent):
    """
    Write segment data to Excel file with separate worksheets for horizontal and vertical segments.
    Creates a new file, overwriting any existing file.
    Args:
        vertical_segments (list): List of vertical segment dictionaries containing segment info.
        horizontal_segments (list): List of horizontal segment dictionaries containing segment info.
        imag_threshold (int or float): The imaginary threshold value.
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold).
        
    Returns:
        None
    """
    excel_file_path='results/segments.xlsx'
    # Determine threshold value for Excel
    if is_imag_threshold_exponent:
        threshold_value = 10**imag_threshold
    else:
        threshold_value = imag_threshold
    
    # Create DataFrames
    vertical_df = segments_to_dataframe(vertical_segments, threshold_value)
    horizontal_df = segments_to_dataframe(horizontal_segments, threshold_value)
    
    # Write to Excel with formatting
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        if not horizontal_df.empty:
            horizontal_df.to_excel(writer, sheet_name='Horizontal', index=False)
            # Apply Excel formatting
            apply_excel_formatting(writer.sheets['Horizontal'], horizontal_df)
            
        if not vertical_df.empty:
            vertical_df.to_excel(writer, sheet_name='Vertical', index=False)
            # Apply Excel formatting
            apply_excel_formatting(writer.sheets['Vertical'], vertical_df)
    
    display_and_log_print(f"Segment data written to {excel_file_path}")


def append_segments_to_excel(vertical_segments, horizontal_segments, imag_threshold, is_imag_threshold_exponent):
    """
    Append segment data to existing Excel file. If file doesn't exist, creates a new one.
    Preserves existing data and adds new segment data.
    
    Args:
        vertical_segments (list): List of vertical segment dictionaries containing segment info.
        horizontal_segments (list): List of horizontal segment dictionaries containing segment info.
        imag_threshold (int or float): The imaginary threshold value.
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold).
        
    Returns:
        None
    """
    excel_file_path='results/segments.xlsx'
    # Determine threshold value for Excel
    if is_imag_threshold_exponent:
        threshold_value = 10**imag_threshold
    else:
        threshold_value = imag_threshold
    
    # Create DataFrames for new data
    new_vertical_df = segments_to_dataframe(vertical_segments, threshold_value)
    new_horizontal_df = segments_to_dataframe(horizontal_segments, threshold_value)
    
    # Read existing data if file exists
    existing_vertical_df = pd.DataFrame()
    existing_horizontal_df = pd.DataFrame()
    
    if os.path.exists(excel_file_path):
        try:
            # Read existing horizontal segments first
            try:
                existing_horizontal_df = pd.read_excel(excel_file_path, sheet_name='Horizontal')
            except (ValueError, KeyError):
                pass  # Sheet doesn't exist yet
            
            # Read existing vertical segments
            try:
                existing_vertical_df = pd.read_excel(excel_file_path, sheet_name='Vertical')
            except (ValueError, KeyError):
                pass  # Sheet doesn't exist yet
        except Exception as e:
            display_and_log_print(f"Warning: Could not read existing Excel file: {e}")
    
    # Combine existing and new data
    if not existing_horizontal_df.empty and not new_horizontal_df.empty:
        combined_horizontal_df = pd.concat([existing_horizontal_df, new_horizontal_df], ignore_index=True)
    elif not new_horizontal_df.empty:
        combined_horizontal_df = new_horizontal_df
    else:
        combined_horizontal_df = existing_horizontal_df
    
    if not existing_vertical_df.empty and not new_vertical_df.empty:
        combined_vertical_df = pd.concat([existing_vertical_df, new_vertical_df], ignore_index=True)
    elif not new_vertical_df.empty:
        combined_vertical_df = new_vertical_df
    else:
        combined_vertical_df = existing_vertical_df
    
    # Sort the combined DataFrames before writing to Excel
    if not combined_horizontal_df.empty:
        combined_horizontal_df = combined_horizontal_df.sort_values(
            by=['Threshold', 'Value/π Start'], 
            ascending=[True, False], 
            ignore_index=True
        )
    
    if not combined_vertical_df.empty:
        combined_vertical_df = combined_vertical_df.sort_values(
            by=['Threshold', 'Value/π Start'], 
            ascending=[True, False], 
            ignore_index=True
        )
    
    # Write to Excel with formatting
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        if not combined_horizontal_df.empty:
            combined_horizontal_df.to_excel(writer, sheet_name='Horizontal', index=False)
            apply_excel_formatting(writer.sheets['Horizontal'], combined_horizontal_df)
            
        if not combined_vertical_df.empty:
            combined_vertical_df.to_excel(writer, sheet_name='Vertical', index=False)
            apply_excel_formatting(writer.sheets['Vertical'], combined_vertical_df)
    
    display_and_log_print(f"Segment data appended to {excel_file_path}")

In [ ]:
def generate_segment_markdown_table(vertical_segments, horizontal_segments,
                                    imag_threshold, is_imag_threshold_exponent):
    """
    Generate Markdown tables for vertical and horizontal segments, with threshold noted.

    Args:
        vertical_segments (list): List of vertical segment dictionaries containing segment info.
        horizontal_segments (list): List of horizontal segment dictionaries containing segment info.
        imag_threshold (int or float): The imaginary threshold value. If is_imag_threshold_exponent is True,
                                      it's an exponent (int), otherwise, it's the direct threshold value (float).
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold),
                                           if False, treat it as the direct threshold value.
    
    Returns:
        str: Combined markdown tables for both vertical and horizontal segments.
    """
    if is_imag_threshold_exponent:
        title_threshold_str = f"10^{{{imag_threshold}}}"
    else:
        title_threshold_str = f"{imag_threshold*100:.2f}%"

    full_table = create_segment_table(vertical_segments, "vertical", title_threshold_str)
    full_table += create_segment_table(horizontal_segments, "horizontal", title_threshold_str)
    
    return full_table



### Hardware Calibration and Threshold Selection

The `classical_like_segments` function plays a crucial role in **hardware calibration and threshold selection** for coherent error detection. This function processes the pseudo-entropy data to identify and analyze "classical-like" regions within the parameter space. These regions are characterized by a positive real part and a very small imaginary component of the pseudo-entropy, indicating that the quantum system behaves predictably, similar to a classical system, under specific conditions.

The primary purpose of this analysis is to:

1.  **Identify Stable Operating Points:** Pinpoint regions where coherent errors are minimal or where the system's response is well-behaved, which can inform optimal operating parameters for quantum hardware.
2.  **Establish Error Detection Thresholds:** By analyzing the boundaries between "classical-like" and "quantum" (error-affected) regions, we can define practical thresholds for detecting coherent errors. The function allows for both direct numerical thresholds and exponent-based thresholds (e.g., $10^{-X}$), providing flexibility in defining sensitivity.
3.  **Quantify Robustness:** The analysis of continuous segments within these classical-like regions, along with Clopper-Pearson confidence intervals, provides a statistically robust measure of the system's resilience to coherent errors. Longer and more robust segments indicate greater stability against variations in $\beta$ and $\delta$.
4.  **Inform Calibration Strategies:** Understanding the landscape of classical-like behavior helps in designing more efficient calibration protocols, focusing efforts on regions where the system is most sensitive to coherent errors or where fine-tuning yields the greatest improvement in fidelity.

The function iterates through a range of imaginary thresholds (or a list of specific thresholds) to generate:

* **Binary Plots:** Visualizations that clearly delineate classical-like, quantum, and undefined (NaN) regions.
* **Segment Analysis:** Identification and characterization of continuous segments of classical-like behavior.
* **Markdown Tables:** Detailed statistical summaries of these segments, including their length, average "true" percentage, and confidence intervals.
* **Excel Export:** All segment data is systematically saved to an Excel file for further external analysis and reporting.

This systematic approach provides a comprehensive diagnostic tool for characterizing and mitigating coherent errors in quantum computing systems.

In [ ]:
def create_classical_quantum_plot(data_to_plot, plot_extent, imag_threshold, is_imag_threshold_exponent, title_suffix="", filename_suffix=""):
    """
    Helper function to create and save a binary plot of classical-like vs quantum regions.

    Args:
        data_to_plot (bool ndarray): The boolean mask to plot (True for classical-like).
        plot_extent (list): The plot boundaries [x_min, x_max, y_min, y_max].
        imag_threshold (int or float): The imaginary threshold. If is_imag_threshold_exponent is True,
                                      it's an exponent (int), otherwise, it's the direct threshold value (float).
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold),
                                           if False, treat it as the direct threshold value.
        title_suffix (str, optional): Additional text to append to the plot title. Defaults to "".
        filename_suffix (str, optional): Additional text to append to the filename. Defaults to "".
    """
    if is_imag_threshold_exponent:
        threshold_label = f"{np.abs(imag_threshold):.0f}"
        title_threshold_str = f"10^{{{imag_threshold}}}"
        filename_threshold_str = f"{np.abs(imag_threshold)}"
    else:
        threshold_label = f"{imag_threshold*100:.2f}"
        title_threshold_str = f"{threshold_label}\\%"
        filename_threshold_str = threshold_label.replace(".", "_").replace("-", "")

    binary_cmap = mcolors.ListedColormap(['black', 'white'])

    plt.figure(figsize=(12, 10), dpi=300)
    plt.imshow(data_to_plot, extent=plot_extent, origin='lower', aspect='auto', cmap=binary_cmap)

    cbar = plt.colorbar(ticks=[0.25, 0.75])
    cbar.ax.set_yticklabels(['Quantum Region', 'Classical-like Region'])

    plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
    plt.xlabel(r'Controlled Interaction Strength $\beta / \pi$', fontsize=14)
    plt.ylabel(r'Coherent Error Angle $\delta / \pi$', fontsize=14)
    plt.title(
        rf'Classical-like vs Quantum Behavior Regions of Pseudo-Entropy $\check{{S}}$ (Threshold: $|Im(\check{{S}})| < {title_threshold_str}$)'
        + title_suffix,
        fontsize=16
    )
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    filename = f'results/classical_vs_quantum_regions_thresh_{filename_threshold_str}{filename_suffix}.png'
    plt.savefig(filename, bbox_inches='tight')
    plt.show()

    latex_import = (
        "\\begin{figure}[ht]\n"
        "    \\centering\n"
        f"    \\includegraphics[width=0.8\\textwidth]{{{filename}}}\n"
        f"    \\caption{{Classical-like vs Quantum Behavior Regions of Pseudo-Entropy $\\check{{S}}$ "
        f"with $|\\Im(\\check{{S}})| < {title_threshold_str}$" + title_suffix + "}\n"
        f"    \\label{{fig:classical_quantum_regions_{filename_threshold_str.replace('.', '_').replace('-', '')}{filename_suffix.replace('-', '_')}}}\n"
        "\\end{figure}"
    )
    display_and_log_print(latex_import)


def plot_classical_like_quantum(betas, deltas, data, imag_threshold, is_imag_threshold_exponent, zoom_factor=None):
    """
    Generate a binary visualization highlighting regions where pseudo-entropy behaves more
    classically (positive real part and small imaginary component) versus purely quantum regions,
    while also identifying undefined (NaN) regions. Optionally, create additional zoomed-in views.

    Args:
        betas (array): Array of measurement angles.
        deltas (array): Array of coherent error angles.
        data (complex ndarray): Complex array containing pseudo-entropy or related values to plot.
        imag_threshold (int or float): The imaginary threshold. If is_imag_threshold_exponent is True,
                                      it's an exponent (int), otherwise, it's the direct threshold value (float).
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold),
                                           if False, treat it as the direct threshold value.
        zoom_factor (float, optional): Factor to zoom in on central region of the plot. If None, no zoom is applied.

    Returns:
        numpy.ndarray: A boolean mask with shape (len(deltas), len(betas)), where the
                       rows correspond to the values in the deltas array and the
                       columns correspond to the values in the betas array.
                       True indicates classical-like regions and False indicates
                       quantum regions for the corresponding ($\\delta$, $\\beta$) parameter pair.

    Description:
        This function creates one or more visualizations with three distinct regions:
        - **White regions**: Classical-like behavior (Re(Ŝ) > 0 and |Im(Ŝ)| < threshold)
        - **Black regions**: Quantum behavior (conditions not met)
        - **Gray regions**: Undefined/NaN regions (calculation failed)

        If zoom_factor is provided, an additional plot focusing on the central portion is generated.
    """
    # Threshold handling
    if is_imag_threshold_exponent:
        threshold = 10 ** imag_threshold
        threshold_label = f"10^{{{imag_threshold}}}"
    else:
        threshold = imag_threshold
        threshold_label = f"{imag_threshold*100:.2f}\\%"

    # Create masks
    nan_mask = np.isnan(data.real) | np.isnan(data.imag)
    classical_mask = (~nan_mask) & (data.real > 0) & (np.abs(data.imag) < threshold)
    quantum_mask = (~nan_mask) & ~classical_mask

    # Define plot extents
    extent = [betas[0] / np.pi, betas[-1] / np.pi, deltas[0] / np.pi, deltas[-1] / np.pi]

    # Create plots
    create_classical_quantum_plot(classical_mask, extent, imag_threshold, is_imag_threshold_exponent)

    # Zoomed plot handling (unchanged)
    if zoom_factor is not None:
        center_delta_index = len(deltas) // 2
        center_beta_index = len(betas) // 2

        # Invert zoom logic: larger zoom_factor means smaller view window
        view_window_factor = 1.0 / zoom_factor

        delta_half_range = int((view_window_factor * len(deltas)) // 2)
        beta_half_range = int((view_window_factor * len(betas)) // 2)

        delta_start = max(0, center_delta_index - delta_half_range)
        delta_end = min(len(deltas), center_delta_index + delta_half_range)
        beta_start = max(0, center_beta_index - beta_half_range)
        beta_end = min(len(betas), center_beta_index + beta_half_range)

        zoomed_classical_mask = classical_mask[delta_start:delta_end, beta_start:beta_end]
        zoomed_deltas = deltas[delta_start:delta_end]
        zoomed_betas = betas[beta_start:beta_end]
        zoomed_extent = [zoomed_betas[0] / np.pi, zoomed_betas[-1] / np.pi,
                         zoomed_deltas[0] / np.pi, zoomed_deltas[-1] / np.pi]

        # Add zoom factor to the title
        zoom_title_suffix = f" (Zoomed {zoom_factor}x)"
        zoom_filename_suffix = f"_zoom_{zoom_factor}x"
        create_classical_quantum_plot(
            zoomed_classical_mask,
            zoomed_extent,
            imag_threshold,
            is_imag_threshold_exponent,
            zoom_title_suffix,
            zoom_filename_suffix
        )

    # Enhanced statistics with NaN percentage
    total_points = data.size
    classical_points = np.sum(classical_mask)
    quantum_points = np.sum(quantum_mask)
    nan_points = np.sum(nan_mask)
    
    classical_percentage = (classical_points / total_points) * 100
    quantum_percentage = (quantum_points / total_points) * 100
    nan_percentage = (nan_points / total_points) * 100

    # Enhanced markdown table with NaN
    latex_table_stats = (
        "$$\n"
        "\\begin{array}{|l|c|c|} \\hline "
        "\\textbf{Region Type} & \\textbf{Point Count} & \\textbf{Percentage (\\%)} \\\\ \\hline "
        f"\\text{{Classical-like}} & {classical_points} & {classical_percentage:.2f} \\\\ \\hline "
        f"\\text{{Quantum}} & {quantum_points} & {quantum_percentage:.2f} \\\\ \\hline "
        f"\\text{{Undefined (NaN)}} & {nan_points} & {nan_percentage:.2f} \\\\ \\hline "
        f"\\textbf{{Total}} & \\textbf{{{total_points}}} & \\textbf{{100.00}} \\\\ \\hline "
        "\\end{array}\n"
        "$$"
    )

    # Enhanced informational text
    info_text = (
        r"**Region Legend:**"
        "\n\n"
        r"- **White:** Classical-like ($\Re(\check{S}) > 0$ AND $|\Im(\check{S})| < " + f"{threshold_label}$)"
        "\n"
        r"- **Black:** Quantum regions"
        "\n"
        r"- **Gray:** Undefined/NaN (calculation failed)"
        "\n\n"
        r"**Note:** NaN regions occur where initial and final states are orthogonal (post-selection probability ≈ 0)"
    )

    display_and_log_markdown(info_text)
    display_and_log_markdown(latex_table_stats)

    return classical_mask


In [ ]:
def classical_like_segments(betas, deltas, numerical_entropies, threshold_source, is_imag_threshold_exponent, confidence_level=0.95):
    """
    Iterates through a range of imaginary thresholds (or a list of thresholds) and generates plots,
    segment analysis, and Markdown tables, including confidence intervals for segment proportions.

    Args:
        betas (array): Array of measurement angles.
        deltas (array): Array of coherent error angles.
        numerical_entropies (complex ndarray): Complex array containing pseudo-entropy or related values.
        threshold_source (range or list or float): If is_imag_threshold_exponent is True, this is a range of exponents.
                                          If False, this is a list of floats (or a single float) representing the direct threshold.
        is_imag_threshold_exponent (bool): If True, treat imag_threshold as an exponent (10^imag_threshold),
                                           if False, treat it as the direct threshold value.
        confidence_level (float, optional): The confidence level for segment proportion CIs (e.g., 0.95 for 95%).
    """
    if isinstance(threshold_source, (range, list, tuple)):
        threshold_values_to_iterate = threshold_source
    else:
        threshold_values_to_iterate = [threshold_source]  # Wrap single value in list for iteration

    for imag_threshold in threshold_values_to_iterate:
        # Run function with your data
        classical_mask = plot_classical_like_quantum(
            betas, deltas, numerical_entropies, imag_threshold, is_imag_threshold_exponent, 4
        )

        # Analyze vertical and horizontal segments, passing the confidence_level
        vertical_segments = analyze_segments(
            classical_mask, betas, deltas, white_percentage_threshold=0.3, is_vertical=True, confidence_level=confidence_level
        )

        horizontal_segments = analyze_segments(
            classical_mask, betas, deltas, white_percentage_threshold=0.3, is_vertical=False, confidence_level=confidence_level
        )

        # Generate and display Markdown table
        markdown_table = generate_segment_markdown_table(
            vertical_segments, horizontal_segments, imag_threshold, is_imag_threshold_exponent
        )
        display_and_log_markdown(markdown_table)


        # Write to Excel file
        append_segments_to_excel(
            vertical_segments, horizontal_segments, imag_threshold, is_imag_threshold_exponent
        )

        # Visualize segments
        visualize_segments(
            classical_mask, betas, deltas, vertical_segments, horizontal_segments,
            imag_threshold, is_imag_threshold_exponent, 4
        )

### Using a range of exponents

This section initiates the analysis of classical-like segments by iterating through a predefined range of imaginary thresholds, expressed as exponents of 10. This approach allows for a systematic exploration of how the definition of "classical-like" behavior (i.e., the tolerance for the imaginary component of pseudo-entropy) impacts the identification and characterization of stable operating regions. The results from this iterative analysis are visualized in binary plots, summarized in Markdown tables, and exported to Excel for detailed review, providing insights into the robustness and sensitivity of the pseudo-entropy method across different error tolerances.

In [ ]:
# Using a range of exponents
display_and_log_print("\n--- Running analysis for a range of exponents ---")
classical_like_segments(betas, deltas, numerical_entropies, range(-5, -1), True)

### Using CSV Data for Threshold Values

This section extends the analysis of classical-like segments by incorporating threshold values directly from a CSV file. This allows for the use of empirically derived or hardware-specific error rates as thresholds for identifying classical-like behavior. By leveraging external data sources, this approach enables a more practical and context-aware assessment of stable operating regions and error detection capabilities. The process involves reading the 'Total_Readout_Error_Percent' from the `optimal_qubit_groups.csv` file, converting these percentages to decimal thresholds, and then applying them to the `classical_like_segments` function. This provides a direct link between experimental hardware characteristics and the theoretical framework of pseudo-entropy for coherent error detection.

In [ ]:
# Using CSV data for a single threshold
display_and_log_print("\n--- Running analysis with threshold from CSV data ---")
csv_thresholds = []
csv_is_imag_threshold_exponent = False # Assume direct value from CSV

try:
    with open('hardware/optimal_qubit_groups.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if 'Total_Readout_Error_Percent' in row and row['Total_Readout_Error_Percent']:
                readout_error_str = row['Total_Readout_Error_Percent'].replace('%', '')
                try:
                    csv_imag_threshold = float(readout_error_str) / 100.0 # Convert to decimal
                    csv_thresholds.append(csv_imag_threshold)
                except ValueError:
                    display_and_log_print(f"Warning: Could not convert 'Total_Readout_Error_Percent' = '{row['Total_Readout_Error_Percent']}' to float from CSV. Skipping this row.")
except FileNotFoundError:
    display_and_log_print("Warning: CSV file 'results/optimal_qubit_groups.csv' not found. Cannot use CSV threshold.")
except KeyError:
    display_and_log_print("Warning: Column 'Total_Readout_Error_Percent' not found in CSV. Cannot use CSV threshold.")

if csv_thresholds:
    classical_like_segments(betas, deltas, numerical_entropies, csv_thresholds, csv_is_imag_threshold_exponent)
else:
    display_and_log_print("No valid CSV thresholds found, skipping CSV-based plot.")



### Unpacking Parameter Sensitivity and Partial Derivatives

This section explores the sensitivity of the pseudo-entropy to changes in the $\beta$ and $\delta$ parameters by calculating and visualizing their partial derivatives. These "sensitivity maps" reveal which parameters have the most significant impact on the pseudo-entropy's behavior, guiding intuition for parameter selection and highlighting areas where fine-tuning is most effective. The *detailed sub-subsection analysis of sensitivity methods* and the *full phase diagram plots* (if distinct from the sensitivity maps) are offloaded to the supplementary materials for in-depth review.

In [ ]:
def calculate_sensitivity(
    numerical_entropies, 
    betas, 
    deltas,  
    segment_info, 
    is_vertical=True,
    zero_threshold=1e-3, 
    proximity_factor=0.5,
    log_base=np.e,
    epsilon=1e-15  
):
    """
    Calculate sensitivity of imaginary part near classical-like regions for either vertical or horizontal segments.
    
    Parameters:
    -----------
    numerical_entropies : numpy.ndarray
        2D array of imaginary part of pseudo-entropy
    betas : numpy.ndarray
        Beta values corresponding to columns
    deltas : numpy.ndarray
        Delta values corresponding to rows
    segment_info : list of dict
        Pre-existing segments information
    is_vertical : bool, optional
        If True, analyze vertical segments; if False, analyze horizontal segments
    zero_threshold : float, optional
        Threshold for considering values "near zero"
    proximity_factor : float, optional
        Factor to determine proximity to segment gaps
    log_base : float, optional
        Base for logarithmic sensitivity calculation
    epsilon : float, optional
        Small value to prevent log(0)
    
    Returns:
    --------
    sensitivity_info : list of dict
        Sensitivity information for each classical-like region
    """
    sensitivity_info = []

    # Sort segments by start position
    if is_vertical:
        # For vertical segments, sort by beta (column) position
        axis_start_key = 'start_pi'
        axis_end_key = 'end_pi'
        primary_axis_values = betas
    else:
        # For horizontal segments, sort by delta (row) position
        axis_start_key = 'start_pi'
        axis_end_key = 'end_pi'
        primary_axis_values = deltas
        # For horizontal analysis, transpose the entropy array
        numerical_entropies = numerical_entropies.T

    # Sort segments by their start position
    sorted_segments = sorted(segment_info, key=lambda s: s[axis_start_key])

    for i, segment in enumerate(sorted_segments):
        segment_start_pi = segment[axis_start_key]
        segment_end_pi = segment[axis_end_key]

        # Compute gaps separately
        prev_segment_end = sorted_segments[i - 1][axis_end_key] if i > 0 else None
        next_segment_start = sorted_segments[i + 1][axis_start_key] if i < len(sorted_segments) - 1 else None

        gap_prev = segment_start_pi - prev_segment_end if prev_segment_end is not None else np.inf
        gap_next = next_segment_start - segment_end_pi if next_segment_start is not None else np.inf

        # Apply separate proximity factors
        proximity_range_prev = proximity_factor * gap_prev
        proximity_range_next = proximity_factor * gap_next

        # Find the central index for this segment
        central_index = np.argmin(np.abs(primary_axis_values / np.pi - (segment_start_pi + segment_end_pi) / 2))
        central_value = primary_axis_values[central_index] / np.pi

        # Extract the line of imaginary entropy for this position
        imaginary_line = np.abs(numerical_entropies[:, central_index])

        # Find indices close to zero
        near_zero_indices = np.where(imaginary_line < zero_threshold)[0]

        # Calculate the valid range in primary axis space (with proximity)
        start_with_proximity = segment_start_pi - proximity_range_prev
        end_with_proximity = segment_end_pi + proximity_range_next

        # Map these to indices in the primary axis array
        valid_indices = np.where(
            (primary_axis_values / np.pi >= start_with_proximity) & 
            (primary_axis_values / np.pi <= end_with_proximity)
        )[0]

        # Filter the near_zero_indices based on the valid indices
        valid_near_zero_indices = np.intersect1d(near_zero_indices, valid_indices)

        # Ensure enough valid points for meaningful sensitivity calculation
        if len(valid_near_zero_indices) < 2:
            continue  # Skip segment if insufficient points

        # Extract absolute imaginary part only
        imag_part = np.abs(np.imag(numerical_entropies[valid_near_zero_indices, central_index]))

        # Apply log without division and ensure that values are greater than epsilon
        log_sensitivity = np.full_like(imag_part, np.nan, dtype=float)

        # Only compute log for values greater than epsilon
        log_mask = imag_part > epsilon
        log_sensitivity[log_mask] = np.log(imag_part[log_mask]) / np.log(log_base)

        # Create a result dictionary with all the info
        result = {
            'central_value_pi': central_value,
            'log_sensitivity': log_sensitivity,
            'valid_indices': valid_near_zero_indices,
            'proximity_range_prev': proximity_range_prev,
            'proximity_range_next': proximity_range_next,
            'is_vertical': is_vertical,
            **segment  # Preserve all original segment information
        }
        
        # Add specific keys based on orientationß
        if is_vertical:
            result['central_beta_pi'] = central_value
        else:
            result['central_delta_pi'] = central_value
            
        sensitivity_info.append(result)

With our sensitivities calculated, let's see how these calculations can show up visually to guide our intution! We will use these images to help improve how parameters are selected, and make changes in the right direction!

In [ ]:
def create_symmetric_log_norm(data):
    """Creates a symmetric log norm with improved separation around zero."""
    # Filter out non-finite values (NaN, inf) before calculations
    finite_data = data[np.isfinite(data)]
    
    if finite_data.size == 0:
        display_and_log_print("Warning: No finite data for normalization. Using default normalization.")
        return None
    
    vabs_max = np.abs(finite_data).max()
    
    # Check for constant data
    if vabs_max == 0:
        display_and_log_print("Warning: All finite data is zero. Using default normalization.")
        return None
    
    # Use finite data for median calculation
    data_median = np.median(np.abs(finite_data))
    
    # Ensure median is not zero (which could cause issues)
    if data_median == 0:
        # Use a small fraction of the max value instead
        data_median = vabs_max * 0.01
    
    try:
        return mcolors.SymLogNorm(linthresh=data_median, linscale=data_median,
                                vmin=-vabs_max, vmax=vabs_max)
    except ValueError as e:
        display_and_log_print(f"Warning: Failed to create SymLogNorm: {e}. Using default normalization.")
        return None

def display_heatmap(data, x_coords, y_coords, title, x_label, y_label,
                    color_map='viridis', normalization=None, filename=None,
                    results_dir="results", cbar_label=None):
    """Displays a heatmap with consistent formatting and saves it to the specified folder."""

    # Check for all-NaN data
    if np.all(np.isnan(data)):
        display_and_log_print("Warning: All data is NaN. Skipping heatmap plot.")
        return None

    # Check for constant data (excluding NaNs)
    finite_data = data[np.isfinite(data)]
    if finite_data.size == 0:
        display_and_log_print("Warning: No finite data. Skipping heatmap plot.")
        return None
    if np.nanmax(data) == np.nanmin(data):
        display_and_log_print("Warning: Data is constant. Heatmap may not be informative.")
        # Set normalization to None for constant data
        normalization = None

    figure, axes = plt.subplots(figsize=(12, 9))
    masked_data = np.ma.masked_invalid(data)
    
    try:
        image = axes.imshow(masked_data, cmap=color_map, norm=normalization, aspect='auto')
    except ValueError as e:
        display_and_log_print(f"Warning: imshow failed with normalization ({e}). Using default normalization.")
        image = axes.imshow(masked_data, cmap=color_map, norm=None, aspect='auto')

    axes.set_xlabel(x_label)
    axes.set_ylabel(y_label)
    axes.set_title(title)

    num_x_ticks = 10
    num_y_ticks = 10
    x_tick_indices = np.linspace(0, len(x_coords) - 1, num_x_ticks).astype(int)
    y_tick_indices = np.linspace(0, len(y_coords) - 1, num_y_ticks).astype(int)

    axes.set_xticks(x_tick_indices)
    axes.set_yticks(y_tick_indices)
    axes.set_xticklabels(np.round(x_coords[x_tick_indices], 2), rotation=45, ha='right')
    axes.set_yticklabels(np.round(y_coords[y_tick_indices], 3))

    try:
        color_bar = plt.colorbar(image, ax=axes, orientation="horizontal")
        color_bar.ax.tick_params(rotation=45, labelsize=8, pad=15)
        if cbar_label:
            color_bar.set_label(cbar_label)
    except ValueError as e:
        display_and_log_print(f"Warning: Colorbar creation failed ({e}). Plot created without colorbar.")

    plt.tight_layout()

    if filename:
        os.makedirs(results_dir, exist_ok=True)
        filepath = os.path.join(results_dir, filename)
        plt.savefig(filepath)
    plt.show()
    plt.close(figure)
    return filename.replace(".png", "") if filename else None

def generate_latex_figure_tag(filename_base, caption, results_dir="results"):
    """Generates a LaTeX figure environment string."""
    filename = f"{filename_base}.png"
    label = f"fig:{filename_base.replace('.', '_')}"  # Make label LaTeX-friendly
    latex_tag = (
        f"\\begin{{figure}}[htbp]\n"
        f"    \\centering\n"
        f"    \\includegraphics[width=\\textwidth]{{results/{filename}}}\n"
        f"    \\caption{{{caption}}}\n"
        f"    \\label{{{label}}}\n"
        f"\\end{{figure}}\n"
    )
    return latex_tag

def plot_pseudo_entropy_derivatives(entropies, beta_step, delta_step, betas, deltas, results_dir="results"):
    """
    Plots the absolute value and differentiation of the imaginary and real parts
    of pseudo entropy and saves the figures to the specified folder.
    The x and y axes represent the angles divided by pi (dimensionless).
    The derivatives have units of bits.

    Args:
        entropies (np.ndarray): 2D array of pseudo-entropy values (complex, bits).
        beta_step (float): Step size in beta (radians).
        delta_step (float): Step size in delta (radians).
        betas (np.ndarray): 1D array of beta values (radians).
        deltas (np.ndarray): 1D array of delta values (radians).
        results_dir (str): Directory to save the plots.
    """
    beta_step_norm = beta_step / np.pi
    delta_step_norm = delta_step / np.pi
    betas_norm = betas / np.pi
    deltas_norm = deltas / np.pi

    imaginary_abs = np.abs(entropies.imag)  # Units: bits
    real_part_abs = np.abs(entropies.real)   # Units: bits

    d_imag_abs_d_delta = np.gradient(imaginary_abs, delta_step_norm, axis=0)  # Units: bits
    d_imag_abs_d_beta = np.gradient(imaginary_abs, beta_step_norm, axis=1)    # Units: bits
    d_real_abs_d_delta = np.gradient(real_part_abs, delta_step_norm, axis=0)  # Units: bits
    d_real_abs_d_beta = np.gradient(real_part_abs, beta_step_norm, axis=1)    # Units: bits

    latex_tags = []

    # Plot |dIm/dBeta|
    filename = "d_abs_imag_pseudo_entropy_d_beta.png"
    plot_title = r"Imaginary Component Sensitivity to $\beta$: $\left|\frac{\partial \Im(\check{S})}{\partial \beta}\right|$"
    plot_caption = r"Absolute value of partial derivative of imaginary pseudo entropy with respect to $\beta$. This shows the sensitivity of the imaginary component to changes in controlled interaction strength, measured in bits."
    display_heatmap(d_imag_abs_d_beta, betas_norm, deltas_norm,
                    plot_title,
                    r"Controlled Interaction Strength $\beta / \pi$", r"Coherent Error Angle $\delta / \pi$", color_map='coolwarm',
                    normalization=create_symmetric_log_norm(d_imag_abs_d_beta),
                    filename=filename, results_dir=results_dir, cbar_label='Magnitude (bits)')
    latex_tags.append(generate_latex_figure_tag(filename.replace(".png", ""), plot_caption, results_dir=results_dir))

    # Plot |dIm/dDelta|
    filename = "d_abs_imag_pseudo_entropy_d_delta.png"
    plot_title = r"Imaginary Component Sensitivity to $\delta$: $\left|\frac{\partial \Im(\check{S})}{\partial \delta}\right|$"
    plot_caption = r"Absolute value of partial derivative of imaginary pseudo entropy with respect to $\delta$. This shows the sensitivity of the imaginary component to changes in coherent error angle, measured in bits."
    display_heatmap(d_imag_abs_d_delta, betas_norm, deltas_norm,
                    plot_title,
                    r"Controlled Interaction Strength $\beta / \pi$", r"Coherent Error Angle $\delta / \pi$", color_map='coolwarm',
                    normalization=create_symmetric_log_norm(d_imag_abs_d_delta),
                    filename=filename, results_dir=results_dir, cbar_label='Magnitude (bits)')
    latex_tags.append(generate_latex_figure_tag(filename.replace(".png", ""), plot_caption, results_dir=results_dir))

    # Plot |dRe/dBeta|
    filename = "d_abs_real_pseudo_entropy_d_beta.png"
    plot_title = r"Real Component Sensitivity to $\beta$: $\left|\frac{\partial \Re(\check{S})}{\partial \beta}\right|$"
    plot_caption = r"Absolute value of partial derivative of real pseudo entropy with respect to $\beta$. This shows the sensitivity of the real component to changes in controlled interactions strength, measured in bits."
    display_heatmap(d_real_abs_d_beta, betas_norm, deltas_norm,
                    plot_title,
                    r"Controlled Interaction Strength $\beta / \pi$", r"Coherent Error Angle $\delta / \pi$", color_map='coolwarm',
                    normalization=create_symmetric_log_norm(d_real_abs_d_beta),
                    filename=filename, results_dir=results_dir, cbar_label='Magnitude (bits)')
    latex_tags.append(generate_latex_figure_tag(filename.replace(".png", ""), plot_caption, results_dir=results_dir))

    # Plot |dRe/dDelta|
    filename = "d_abs_real_pseudo_entropy_d_delta.png"
    plot_title = r"Real Component Sensitivity to $\delta$: $\left|\frac{\partial \Re(\check{S})}{\partial \delta}\right|$"
    plot_caption = r"Absolute value of partial derivative of real pseudo entropy with respect to $\delta$. This shows the sensitivity of the real component to changes in coherent error angle, measured in bits."
    display_heatmap(d_real_abs_d_delta, betas_norm, deltas_norm,
                    plot_title,
                    r"Controlled Interaction Strength $\beta / \pi$", r"Coherent Error Angle $\delta / \pi$", color_map='coolwarm',
                    normalization=create_symmetric_log_norm(d_real_abs_d_delta),
                    filename=filename, results_dir=results_dir, cbar_label='Magnitude (bits)')
    latex_tags.append(generate_latex_figure_tag(filename.replace(".png", ""), plot_caption, results_dir=results_dir))

    display_and_log_print("\nLaTeX Figure Tags:")
    for tag in latex_tags:
        display_and_log_print(tag)

### Visualizing Partial Derivatives of Pseudo-Entropy

To gain a deeper understanding of how the parameters $\beta$ and $\delta$ influence the pseudo-entropy, we'll now calculate and visualize the partial derivatives of the pseudo-entropy with respect to each parameter. These derivatives will reveal the sensitivity of the pseudo-entropy to changes in these parameters.

By calculating and visualizing the derivatives, we can visually identify and pinpoint regions in which parameter adjustment will be the most influential for fine-tuning results.


In [ ]:
plot_pseudo_entropy_derivatives(numerical_entropies, beta_step, delta_step, betas, deltas)

### Analyzing Sensitivity in Pseudo-Entropy

This section delves into analyzing the quantum circuit's behavior using a sensitivity metric. This sensitivity analysis tells us how much the parameters, $\beta$ and $\delta$, need to change to produce a noticeable variation in the pseudo-entropy $\check{S}$. A lower sensitivity value implies that small changes in the parameter lead to significant changes in pseudo-entropy, indicating stronger responsiveness of the system to that parameter.

In [ ]:
def safe_min_max(array):
    """Compute min and max while handling NaN values."""
    if array is None or len(array) == 0:
        return np.nan, np.nan
    
    # Filter out NaN values
    valid_values = array[~np.isnan(array)]
    
    if len(valid_values) == 0:
        return np.nan, np.nan
    
    return np.min(valid_values), np.max(valid_values)

def calculate_sensitivity(betas, deltas, pseudo_entropies, std_deviations):
    """
    Calculates sensitivity for beta and delta parameters based on the
    pseudo-entropy values, their derivatives, and their standard deviations.

    Args:
        betas (np.ndarray): Array of beta values.
        deltas (np.ndarray): Array of delta values.
        pseudo_entropies (np.ndarray): NumPy array of complex pseudo-entropies
                                      (delta as rows, betas as columns).
        std_deviations (np.ndarray): NumPy array of standard deviations for pseudo-entropies
                                    (delta as rows, betas as columns).

    Returns:
        tuple: Sensitivity metrics and arrays for beta and delta.
    """
    # Calculate derivatives using numpy.gradient
    dSdbeta = np.gradient(pseudo_entropies, betas, axis=1)
    dSddelta = np.gradient(pseudo_entropies, deltas, axis=0)
    
    # Use absolute values of derivatives for meaningful sensitivity
    abs_dSdbeta = np.abs(dSdbeta)
    abs_dSddelta = np.abs(dSddelta)
    
    # Use the calculated standard deviations for each point instead of a global uncertainty
    # Calculate sensitivity arrays (pixel-wise) using the formula:
    # Sensitivity_S(θ)^2 = σ_S^2 / |∂S/∂θ|^2
    
    # Add small epsilon to avoid division by zero
    eps = np.finfo(float).eps
    
    # Calculate sensitivity arrays using the proper formula
    sensitivity_beta_array = np.abs(std_deviations) / (abs_dSdbeta + eps)
    sensitivity_delta_array = np.abs(std_deviations) / (abs_dSddelta + eps)
    
    # Calculate summary statistics using NaN-aware functions
    sensitivity_beta = np.nanmedian(sensitivity_beta_array)
    sensitivity_delta = np.nanmedian(sensitivity_delta_array)
    
    # Use the mean standard deviation as the general uncertainty value for reporting
    uncertainty = np.nanmean(np.abs(std_deviations))
    
    return sensitivity_beta, sensitivity_delta, sensitivity_beta_array, sensitivity_delta_array, uncertainty

In [ ]:
def plot_sensitivity(betas, deltas, sensitivity_beta_array, sensitivity_delta_array):
    """
    Plots Sensitivity heatmaps for beta and delta parameters.
    The Sensitivity values have units of radians.
    The x and y axes represent the angles divided by pi (dimensionless).

    Args:
        betas (np.ndarray): Array of beta values (radians).
        deltas (np.ndarray): Array of delta values (radians).
        sensitivity_beta_array (np.ndarray): 2D array of sensitivity values for beta (radians).
        sensitivity_delta_array (np.ndarray): 2D array of sensitivity values for delta (radians).
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Use percentile-based limits for better visualization
    vmin_beta, vmax_beta = np.percentile(sensitivity_beta_array, [5, 95])
    vmin_delta, vmax_delta = np.percentile(sensitivity_delta_array, [5, 95])

    # Create heatmaps for sensitivity
    im1 = axes[0].imshow(sensitivity_beta_array,
                         extent=[betas.min() / np.pi, betas.max() / np.pi,
                                 deltas.min() / np.pi, deltas.max() / np.pi],
                         origin='lower', aspect='auto', cmap='viridis',
                         vmin=vmin_beta, vmax=vmax_beta)
    axes[0].set_xlabel(r'Controlled Interaction Strength $\beta / \pi$')
    axes[0].set_ylabel(r'Coherent Error Angle $\delta / \pi$')
    axes[0].set_title('Sensitivity for $\\beta$ (radians)', pad=20)  # Add vertical padding
    fig.colorbar(im1, ax=axes[0], label='(radians)')

    im2 = axes[1].imshow(sensitivity_delta_array,
                         extent=[betas.min() / np.pi, betas.max() / np.pi,
                                 deltas.min() / np.pi, deltas.max() / np.pi],
                         origin='lower', aspect='auto', cmap='viridis',
                         vmin=vmin_delta, vmax=vmax_delta)
    axes[1].set_xlabel(r'Controlled Interaction Strength $\beta / \pi$')
    axes[1].set_ylabel(r'Coherent Error Angle $\delta / \pi$')
    axes[1].set_title('Sensitivity for $\\delta$ (radians)', pad=20)
    fig.colorbar(im2, ax=axes[1], label='(radians)')

    plt.tight_layout()
    plt.subplots_adjust(top=0.88)  # Adjusts space at the top if needed
    plt.savefig("results/variables_sensitivity.png")
    plt.show()

In [ ]:
def safe_min_max(array):
    """Compute min and max while handling NaN values."""
    if array is None or len(array) == 0:
        return np.nan, np.nan
    
    # Filter out NaN values
    valid_values = array[~np.isnan(array)]
    
    if len(valid_values) == 0:
        return np.nan, np.nan
    
    return np.min(valid_values), np.max(valid_values)
    
def sensitivity_to_dataframe(sensitivity_beta, sensitivity_delta, uncertainty, 
                                  sensitivity_beta_array, sensitivity_delta_array):
    """
    Converts sensitivity and uncertainty data into a pandas DataFrame.
    """
    sensitivity_beta_min, sensitivity_beta_max = safe_min_max(sensitivity_beta_array)
    sensitivity_delta_min, sensitivity_delta_max = safe_min_max(sensitivity_delta_array)

    df_data = [
        {'Metric': 'Global standard deviation of pseudo entropy', 'Symbol': r'$\sigma_{\check{S}}$',
         'Mean Value': uncertainty, 'Min Value': np.nan, 'Max Value': np.nan, 'Units': 'bits'},
        {'Metric': r'Sensitivity over $\beta$', 'Symbol': r'$S_{\beta}$',
         'Mean Value': sensitivity_beta, 'Min Value': sensitivity_beta_min, 'Max Value': sensitivity_beta_max, 'Units': 'radians'},
        {'Metric': r'Sensitivity over $\delta$', 'Symbol': r'$S_{\delta}$',
         'Mean Value': sensitivity_delta, 'Min Value': sensitivity_delta_min, 'Max Value': sensitivity_delta_max, 'Units': 'radians'}
    ]
    return pd.DataFrame(df_data)

def export_sensitivity_to_excel(sensitivity_df, filename='results/sensitivity_results.xlsx'):
    """
    Exports a pandas DataFrame of sensitivity data to an Excel file.
    """
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    writer = pd.ExcelWriter(filename, engine='openpyxl')

    sheet_name = 'Sensitivity_Analysis'
    
    # If the sheet already exists, remove it to prevent duplicate headers
    if sheet_name in writer.book.sheetnames:
        idx = writer.book.sheetnames.index(sheet_name)
        writer.book.remove(writer.book.worksheets[idx])

    sensitivity_df.to_excel(writer, sheet_name=sheet_name, index=False)
    apply_excel_formatting(writer.sheets[sheet_name], sensitivity_df) # Apply formatting
    
    writer.close()
    display_and_log_print(f"Sensitivity data exported to {filename} (Sheet: {sheet_name})")


    
def sensitivity_uncertainty_table(sensitivity_beta, sensitivity_delta, uncertainty, 
                                  sensitivity_beta_array, sensitivity_delta_array):
    """
    Prints a markdown summary table of sensitivity and uncertainty results using LaTeX symbols and including units.
    Also exports the sensitivity and uncertainty data to an Excel file.
    
    Handles NaN values gracefully by filtering them out before computing statistics.

    Args:
        sensitivity_beta (float): Mean sensitivity with respect to beta (units: radians).
        sensitivity_delta (float): Mean sensitivity with respect to delta (units: radians).
        uncertainty (float): Standard deviation of the pseudo-entropy (units: bits).
        sensitivity_beta_array (np.ndarray): Full array of sensitivity values for beta (units: radians).
        sensitivity_delta_array (np.ndarray): Full array of sensitivity values for delta (units: radians).

    Returns:
        None: Displays and logs a summary in Markdown format with LaTeX formatting.
    """
    
    # Compute min and max for sensitivity arrays with NaN handling
    sensitivity_beta_min, sensitivity_beta_max = safe_min_max(sensitivity_beta_array)
    sensitivity_delta_min, sensitivity_delta_max = safe_min_max(sensitivity_delta_array)

    # Prepare the table data with proper NaN handling
    table_data = [
        ["Metric", "Symbol", "Mean Value", "Min Value", "Max Value", "Units"],
        [r"Global standard deviation of pseudo entropy", r"$\sigma_{\check{S}}$",
         to_latex(round_number(uncertainty)), "-", "-", "bits"],
        [r"Sensitivity over $\beta$",
         r"$S_{\beta} = \sigma_{\check{S}} / \|\frac{{\partial \check{{S}}}}{{\partial \beta}}\|$",
         to_latex(round_number(sensitivity_beta)),
         to_latex(round_number(sensitivity_beta_min)),
         to_latex(round_number(sensitivity_beta_max)), "radians"],
        [r"Sensitivity over $\delta$",
         r"$S_{\delta} = \sigma_{\check{S}} / \|\frac{{\partial \check{{S}}}}{{\partial \delta}}\|$",
         to_latex(round_number(sensitivity_delta)),
         to_latex(round_number(sensitivity_delta_min)),
         to_latex(round_number(sensitivity_delta_max)), "radians"]
    ]

    # Create and display the markdown table
    markdown_table = tabulate(table_data, headers="firstrow", tablefmt="pipe")
    display_and_log_markdown("**Sensitivity and Uncertainty Analysis**\n" + markdown_table)

    # Export to Excel
    sensitivity_df = sensitivity_to_dataframe(sensitivity_beta, sensitivity_delta, uncertainty,
                                              sensitivity_beta_array, sensitivity_delta_array)
    export_sensitivity_to_excel(sensitivity_df)

In [ ]:
def print_sensitivity_latex():
    """
    Prints LaTeX code for including Sensitivity figure in a LaTeX document.
    """
    latex_code = r"""
\begin{figure}[htbp]
    \centering
    \includegraphics[width=\textwidth]{results/variables_sensitivity.png}
    \caption{Measurement sensitivity for controlled interaction strength ($\beta$) and coherent error angle ($\delta$) 
    parameters. Lower values (darker blue) indicate higher precision.}
    \label{fig:variables_sensitivity}
\end{figure}
"""
    display_and_log_print("\nLaTeX code for Sensitivity figure:" + latex_code)

In [ ]:
# Calculate Sensitivity
sensitivity_beta, sensitivity_delta, sensitivity_beta_array, sensitivity_delta_array, uncertainty = calculate_sensitivity(betas, deltas, numerical_entropies, numerical_std_deviations)

# Plotting and table output
plot_sensitivity(betas, deltas, sensitivity_beta_array, sensitivity_delta_array)
sensitivity_uncertainty_table(sensitivity_beta, sensitivity_delta, uncertainty, 
                                     sensitivity_beta_array, sensitivity_delta_array)

# Print figure tag
print_sensitivity_latex()